In [ ]:
# "Dmoz_Computers","Dmoz_Health","Dmoz_Science","Dmoz_Sports"
# "classic4","NSF","re8","review_polarity", "Industry_Sector"
# "SyskillWebert","webkb_parsed", "CSTR"
dataset_name = "Industry_Sector"

time: 565 µs (started: 2024-10-16 20:50:09 +00:00)


# Imports, load data , training functions and database creation

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 3.12 s (started: 2024-10-16 20:50:10 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.32 s (started: 2024-10-16 20:50:14 +00:00)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, errno
import networkx as nx
from networkx.algorithms import bipartite
import json
import yaml
import ast
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedShuffleSplit

import pickle
import time

import tensorflow as tf

time: 687 µs (started: 2024-10-16 20:50:16 +00:00)


In [ ]:
#!pip uninstall torch -y

time: 199 µs (started: 2024-10-16 20:50:16 +00:00)


In [ ]:
#!pip install torch==2.1.0

time: 257 µs (started: 2024-10-16 20:50:16 +00:00)


In [ ]:
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
time: 26.5 s (started: 2024-10-16 20:50:16 +00:00)


In [ ]:
import torch_geometric
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.utils.convert import from_networkx
from torch_geometric.nn import SAGEConv, GATConv, Linear, to_hetero

time: 686 µs (started: 2024-10-16 20:50:42 +00:00)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

time: 3.02 ms (started: 2024-10-16 20:50:42 +00:00)


## load data

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

time: 12.4 s (started: 2024-10-16 20:50:42 +00:00)


In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,basic_materials_http:^^www.littonacdipe.com,Litton Litton: Interconnect Prod...,basic_materials,"[(packaging esp, 0.5861), (litton interconnect...","[(electronic systems packaging, 0.6613), (syst...","[(electronic systems packaging, 0.6613), (syst...","[-0.05860152, 0.0145570105, 0.017551647, -0.02...","[[-0.07081439, 0.053046886, 0.038261086, -0.04...","[[-0.07142605, 0.085836, -0.04343102, -0.05472...","[[-0.07142605, 0.085836, -0.04343102, -0.05472...","[[-0.0956507, -0.0512441, -0.009371761, 0.0161..."
1,1,basic_materials_http:^^www.weirton.com^contact...,"Allow: GET, HEAD URI: Contact Weirton St...",basic_materials,"[(contact firm, 0.562), (contact weirton, 0.50...","[(contact weirton steel, 0.5781), (come contac...","[(contact weirton steel, 0.5781), (contact fir...","[-0.124327414, -0.012992704, 0.014001042, 0.03...","[[-0.08809786, -0.019295853, 0.00693113, -0.01...","[[-0.10939312, -0.018659644, 0.051947754, 0.07...","[[-0.10939312, -0.018659644, 0.051947754, 0.07...","[[-0.09154996, 0.004012895, 0.07021415, -0.001..."
2,2,basic_materials_http:^^home.earthlink.net^~nuc...,Systems Which system would you choose?...,basic_materials,"[(pressure cylinders, 0.6841), (pressure bulk,...","[(high pressure cylinders, 0.7351), (choose lo...","[(high pressure cylinders, 0.7351), (pressure ...","[-0.085865036, 0.043530438, -0.071711995, -0.0...","[[-0.097670145, 0.07834313, -0.067915626, -0.0...","[[-0.093530454, 0.07404681, -0.049339183, -0.0...","[[-0.093530454, 0.07404681, -0.049339183, -0.0...","[[-0.03351223, -0.006459103, -0.07732453, -0.0..."
3,3,basic_materials_http:^^www.useg.com^invest.html,US Energy Corporation Investor News ...,basic_materials,"[(images investon, 0.5688), (gif investon, 0.5...","[(images investon gif, 0.6559), (images invest...","[(images investon gif, 0.6559), (images invest...","[0.008732203, -0.013307741, -0.036661223, 0.02...","[[-0.009558729, 0.017956274, -0.059580665, -0....","[[0.00017329463, 0.0013028142, -0.06374502, -0...","[[0.00017329463, 0.0013028142, -0.06374502, -0...","[[0.008732203, -0.013307741, -0.036661223, 0.0..."
4,4,basic_materials_http:^^www.terraindustries.com...,"Terra Industries, Inc., Company Informat...",basic_materials,"[(terra nitrogen, 0.6228), (nitrogen fertilize...","[(terra produces nitrogen, 0.7287), (terra nit...","[(terra produces nitrogen, 0.7287), (terra nit...","[-0.049107973, -0.043558124, -0.020951217, -0....","[[-0.07773064, 0.033809956, 0.025796082, -0.01...","[[-0.045650914, 0.045816682, 0.023411866, -0.0...","[[-0.045650914, 0.045816682, 0.023411866, -0.0...","[[-0.034066442, 0.033516087, 0.018464932, -0.0..."


time: 1.02 s (started: 2024-10-16 20:50:55 +00:00)


## Training functions and Model Definitions

In [ ]:
def creat_equal_masks(test_perc,y):
    n_samples = len(y)
    print('all:',np.bincount(y))
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_perc, random_state=0)
    x = np.zeros(n_samples)
    for train_idx, test_idx in splitter.split(x, y):
        train_mask = np.full(n_samples, False, dtype=bool)
        val_mask = np.full(n_samples, False, dtype=bool)
        test_mask = np.full(n_samples, False, dtype=bool)

        train_mask[train_idx] = True
        test_mask[test_idx] = True

        print('train:',np.bincount(y[train_idx]))
        print('test:',np.bincount(y[test_mask]))

        print('train vs test:',np.bincount(train_mask))
        # print(np.bincount(val_mask))
        # print(np.bincount(test_mask))

        train_mask = torch.tensor(train_mask)
        val_mask = torch.tensor(val_mask)
        test_mask = torch.tensor(test_mask)

        return train_mask, val_mask, test_mask

time: 1.08 ms (started: 2024-10-16 20:50:56 +00:00)


In [ ]:
def train(model,optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['document'].train_mask
    loss = F.cross_entropy(out['document'][mask], data['document'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

time: 674 µs (started: 2024-10-16 20:50:56 +00:00)


In [ ]:
def test(model):
    model.eval()
    out = model(data.x_dict, data.edge_index_dict)
    pred = out['document'].argmax(dim=1)
    # print("PREDICTIONS ->", pred)
    accs = []
    for mask in [data['document'].train_mask, data['document'].test_mask]:
        accs.append(int((pred[mask] == data['document'].y[mask]).sum()) / int(mask.sum()))
    return accs

time: 841 µs (started: 2024-10-16 20:50:56 +00:00)


In [ ]:
def get_y(model):
    model.eval()
    out = model(data.x_dict, data.edge_index_dict)
    pred = out['document'].argmax(dim=1)
    print("PREDICTIONS ->", pred)

    # y_pred, y_true
    return pred[data['document'].test_mask],data['document'].y[data['document'].test_mask]

time: 620 µs (started: 2024-10-16 20:50:56 +00:00)


### GNN Model

In [ ]:
# First Network
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GraphConv(-1, hidden_channels)
        self.conv2 = GraphConv(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

time: 727 µs (started: 2024-10-16 20:50:56 +00:00)


### GAT Model

In [ ]:
# Second Network
from torch_geometric.nn import GATv2Conv

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATv2Conv((-1, -1), hidden_channels, add_self_loops=False)
        self.lin1 = Linear(-1, hidden_channels)
        self.conv2 = GATv2Conv((-1, -1), out_channels, add_self_loops=False)
        self.lin2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = x.relu()
        x = self.conv2(x, edge_index) + self.lin2(x)
        return x

time: 903 µs (started: 2024-10-16 20:50:56 +00:00)


### iteration

In [ ]:
def iteration(i,GCN_or_GAT,rotulated):
    # for i in range(10):
    print(f"\n===============================================")
    print(f"=================== MODEL {i} ===================")
    print(f"===============================================\n")

    # Defining masks
    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks_big/mask_rot'+str(rotulated)+'_'+str(i)+'.pkl', 'rb') as f:
        aux = pickle.load(f)
        train_mask, val_mask, test_mask = aux[0],aux[1],aux[2]

    data['document'].train_mask = train_mask
    data['document'].val_mask = val_mask
    data['document'].test_mask = test_mask

    # Generate MODEL and Optmizer
    if GCN_or_GAT == "GCN":
        model = GNN(hidden_channels=64, out_channels=class_number)
    elif GCN_or_GAT =="GAT":
        model = GAT(hidden_channels=64, out_channels=class_number)

    model = to_hetero(model, data.metadata(), aggr='sum')
    model.to(device) # GPU

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    #contagem de tempo
    start = time.time()

    # Training
    L_loss = []
    for epoch in range(1, 1000):
        loss = train(model,optimizer)
        L_loss.append(loss)
        train_acc, test_acc = test(model)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Test: {test_acc:.4f}')
        df_temp = pd.DataFrame(L_loss)
        if len(df_temp >=5):
          loss_es =  pd.DataFrame(L_loss).tail(5)[0].std()
          print('Early stopping: ',loss_es)
          if loss_es <= 0.01:
            break

    #contagem de tempo
    end = time.time()

    # Creating Classification Report
    y_pred,y_true = get_y(model)
    y_pred = y_pred.cpu()
    y_true = y_true.cpu()
    target_names = df["class"].unique()

    class_dit = classification_report(y_true, y_pred, target_names=target_names, output_dict = True)

    print('\nClassification Report:\n')
    print(classification_report(y_true, y_pred, target_names=target_names))

    #L_results.append([model_name,'GCN',rotulated,class_dit['macro avg']['f1-score']])
    # Updating Dataframes
    df_list[i][0].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['accuracy']               # acuracia
    df_list[i][1].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['precision'] # precision
    df_list[i][2].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['recall']    # recall
    df_list[i][3].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['f1-score']  # f1-score
    df_list[i][4].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = class_dit['macro avg']['support']   # support
    df_list[i][5].at[model_name+"_"+GCN_or_GAT, str(rotulated)] = (end - start)                       # time

time: 1.99 ms (started: 2024-10-16 20:50:56 +00:00)


## Database

In [ ]:
# # Only execude if dafaframes do not exist YET

# df_list = []

# for i in range(10):
#     aux_list = []

#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # acuracia
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # precision
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # recall
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # f1-score
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # support
#     aux_list.append(pd.DataFrame(columns = ['1','5','10','20','30','80%'])) # time

#     df_list.append(aux_list)

#     try:
#         os.makedirs("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i))
#     except OSError as e:
#         if e.errno != errno.EEXIST:
#             raise

#     df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_acc.pkl") # acuracia
#     df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_pre.pkl") # precision
#     df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_rec.pkl") # recall
#     df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_f1.pkl")  # f1-score
#     df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_sup.pkl") # support
#     df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes/"+str(i)+"/df_time.pkl") # time

time: 480 µs (started: 2024-10-16 21:55:36 +00:00)


In [ ]:
df_list = []
for i in range(10):
    aux_list = []
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl")) # acuracia
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl")) # precision
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl")) # recall
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl"))  # f1-score
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl")) # support
    aux_list.append(pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl")) # time
    df_list.append(aux_list)

time: 223 ms (started: 2024-10-16 21:55:37 +00:00)


In [ ]:
df_list[0][0][0:32]

,1,5,10,20,30,80%,20%,40%,60%
Dmoz_Computers_keyphrase2_GCN,0.229301,0.442531,0.514715,0.564583,0.579395,0.693158,0.620658,0.645965,0.685000
Dmoz_Computers_keyphrase2_GAT,0.307984,0.469644,0.540172,0.598684,0.610414,0.725789,0.669605,0.681930,0.701579
Dmoz_Computers_keyphrase3_GCN,0.279823,0.45008,0.534264,0.575,0.603471,0.720000,0.648026,0.672632,0.695000
Dmoz_Computers_keyphrase3_GAT,0.299652,0.489208,0.551128,0.601645,0.615901,0.724211,0.661053,0.683860,0.704211
Dmoz_Computers_keyphrase23_GCN,0.185951,0.417438,0.510311,0.57193,0.58533,0.698421,0.644605,0.671930,0.681842
Dmoz_Computers_keyphrase23_GAT,0.261154,0.480489,0.553383,0.592325,0.610862,0.721053,0.666579,0.681053,0.698947
Dmoz_Health_keyphrase2_GCN,0.262371,0.693862,0.710832,0.747596,0.76743,0.823846,0.816731,0.826154,0.835000
Dmoz_Health_keyphrase2_GAT,0.421921,0.714841,0.754631,0.789744,0.81162,0.861538,0.851154,0.853077,0.860000
Dmoz_Health_keyphrase3_GCN,0.38292,0.681119,0.700157,0.764263,0.787398,0.838462,0.830192,0.835641,0.835385
Dmoz_Health_keyphrase3_GAT,0.418067,0.72805,0.756515,0.797596,0.809493,0.855385,0.850962,0.862564,0.863077


time: 26.2 ms (started: 2024-10-16 21:55:37 +00:00)


In [ ]:
df_list[0][0][32:]

,1,5,10,20,30,80%,20%,40%,60%
NSF_keyphrase3_GCN,0.565188,0.722233,0.76235,0.779008,0.795798,0.858432,0.841686,0.849248,0.858670
NSF_keyphrase3_GAT,0.583746,0.776235,0.770648,0.800666,0.817403,0.852257,0.851544,0.857641,0.858907
NSF_keyphrase23_GCN,0.558622,0.702221,0.758684,0.774794,0.790422,0.843705,0.835986,0.850515,0.856532
NSF_keyphrase23_GAT,0.587172,0.787917,0.777306,0.806644,0.813222,0.866983,0.852969,0.859382,0.859857
classic4_keyphrase2_GCN,0.440558,0.851873,0.913253,0.948396,0.942652,0.958421,0.952431,0.953723,0.959126
classic4_keyphrase2_GAT,0.805951,0.908693,0.894401,0.951675,0.942509,0.965469,0.959126,0.956542,0.957012
classic4_keyphrase3_GCN,0.604851,0.818233,0.873848,0.942124,0.94509,0.955603,0.952607,0.959126,0.961240
classic4_keyphrase3_GAT,0.800028,0.885936,0.871439,0.949679,0.935054,0.964059,0.960183,0.957012,0.961240
classic4_keyphrase23_GCN,0.636582,0.777951,0.808079,0.945403,0.933763,0.967583,0.949260,0.959126,0.955603
classic4_keyphrase23_GAT,0.806656,0.890601,0.898086,0.935139,0.941219,0.963354,0.956483,0.955837,0.960183


time: 30.5 ms (started: 2024-10-16 21:55:37 +00:00)


--------------------------------------
# Hetrogenous Graph for Keyphrase = 2
--------------------------------------

In [ ]:
print(df["class"].unique())
class_number = len(df["class"].unique())
print(class_number)

['basic_materials' 'capital_goods' 'conglomerates_industry'
 'consumer_cyclical' 'consumer_non-cyclical' 'energy' 'financial'
 'healthcare' 'services' 'technology' 'transportation' 'utilities']
12
time: 3.89 ms (started: 2024-10-16 20:50:57 +00:00)


In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase2"

model_name = dataset_name+"_"+keyphrase

time: 444 µs (started: 2024-10-16 20:50:57 +00:00)


## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

document_nodes = []
context_nodes = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

time: 35.2 s (started: 2024-10-16 20:50:57 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 8817
number of context nodes: 31332
number of shared contexts: 11831
number of direct edges (first dimension): 43163
number of direct edges (second dimension): 43163
time: 938 µs (started: 2024-10-16 20:51:32 +00:00)


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

time: 1min 55s (started: 2024-10-16 20:51:32 +00:00)


##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

time: 384 ms (started: 2024-10-16 20:53:27 +00:00)


In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

time: 64.7 ms (started: 2024-10-16 20:53:27 +00:00)


In [ ]:
# Show documents nodes
document_nodes

tensor([[-5.8602e-02,  1.4557e-02,  1.7552e-02,  ..., -1.8911e-02,
         -1.8681e-02, -2.2412e-02],
        [-1.2433e-01, -1.2993e-02,  1.4001e-02,  ..., -1.7944e-02,
          1.1914e-02, -8.1062e-03],
        [-8.5865e-02,  4.3530e-02, -7.1712e-02,  ...,  5.6326e-02,
         -4.5610e-02,  3.1375e-02],
        ...,
        [-1.6584e-02, -3.9770e-02, -2.2835e-02,  ..., -4.2101e-05,
          1.2257e-02,  2.9048e-02],
        [ 3.1864e-02, -1.0648e-02, -3.8576e-02,  ..., -4.5610e-03,
         -1.7202e-02,  4.2433e-02],
        [ 4.3771e-02,  3.8128e-02,  2.0526e-02,  ..., -1.0110e-02,
          3.8847e-02,  4.4935e-02]])

time: 6.2 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# Quantity of document nodes
len(document_nodes)

8817

time: 2.65 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# Show context nodes
context_nodes

tensor([[-0.0708,  0.0530,  0.0383,  ...,  0.0177,  0.0535, -0.0077],
        [-0.0939, -0.0394, -0.0014,  ..., -0.0023,  0.0032, -0.0307],
        [-0.1086,  0.0885, -0.0217,  ...,  0.0218,  0.0936,  0.0187],
        ...,
        [ 0.0020, -0.0129, -0.0552,  ..., -0.0435,  0.0482,  0.0015],
        [-0.0278,  0.0173, -0.0167,  ..., -0.0360,  0.0564,  0.0076],
        [-0.0547,  0.0289,  0.0072,  ...,  0.0046, -0.0190,  0.0114]])

time: 4.61 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# Quantity of context nodes
len(context_nodes)

31332

time: 3.44 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

1946768
1946768
time: 846 µs (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  8816,  8816,  8816],
        [    0,     1,     2,  ..., 23841, 17346, 27373]])
time: 3.11 ms (started: 2024-10-16 20:53:28 +00:00)


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['basic_materials' 'capital_goods' 'conglomerates_industry'
 'consumer_cyclical' 'consumer_non-cyclical' 'energy' 'financial'
 'healthcare' 'services' 'technology' 'transportation' 'utilities']
time: 1.51 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'basic_materials': 0, 'capital_goods': 1, 'conglomerates_industry': 2, 'consumer_cyclical': 3, 'consumer_non-cyclical': 4, 'energy': 5, 'financial': 6, 'healthcare': 7, 'services': 8, 'technology': 9, 'transportation': 10, 'utilities': 11} 



<ipython-input-221-a018e46c97dd>:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  labels["class"][i] = dit[labels["class"][i]]
<ipython-input-221-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,class
0,0
1,0
2,0
3,0
4,0


time: 874 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([ 0,  0,  0,  ..., 11, 11, 11])

time: 4.26 ms (started: 2024-10-16 20:53:28 +00:00)


In [ ]:
len(y)

8817

time: 3.17 ms (started: 2024-10-16 20:53:28 +00:00)


### Testing Graph with Networkx




#### Defining overal graph in networkx maner

In [ ]:
# Run graph Representation for networkx
all_contexts_list =[]
edges_test = []
cont = 0
new_edge_cont = 0

document_nodes_test = []
context_nodes_test = []

# Creating graph nodes and edges
for i in range(len(df)):
    # adding new documents for every node
    document_nodes_test.append("doc_"+str(i)) # in the actual graph nodes -> documents embeddings

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list:
            # add a new context node for every new keyphrase in df
            context_nodes_test.append("contx_"+str(new_edge_cont)) # in the actual graph nodes -> context embeddings

            # add a new edge between doc and new context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(new_edge_cont)))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(all_contexts_list.index(key[0]))))
            cont+=1

time: 13.2 s (started: 2024-10-16 20:53:29 +00:00)


In [ ]:
edges_test = [("doc_"+str(i[0]),"contx_"+str(i[1])) for i in edges_tuple]

time: 1.25 s (started: 2024-10-16 20:53:42 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes_test))
print("number of context nodes:",len(context_nodes_test))
print("number of edges:",len(edges_test))

number of document nodes: 8817
number of context nodes: 31332
number of edges: 1946768
time: 1.1 ms (started: 2024-10-16 20:53:43 +00:00)


In [ ]:
print(edges_test[0:100])

[('doc_0', 'contx_0'), ('doc_0', 'contx_1'), ('doc_0', 'contx_2'), ('doc_0', 'contx_3'), ('doc_0', 'contx_4'), ('doc_1', 'contx_5'), ('doc_1', 'contx_6'), ('doc_1', 'contx_7'), ('doc_1', 'contx_8'), ('doc_1', 'contx_9'), ('doc_2', 'contx_10'), ('doc_2', 'contx_11'), ('doc_2', 'contx_12'), ('doc_2', 'contx_13'), ('doc_2', 'contx_14'), ('doc_3', 'contx_15'), ('doc_3', 'contx_16'), ('doc_3', 'contx_17'), ('doc_3', 'contx_18'), ('doc_3', 'contx_19'), ('doc_4', 'contx_20'), ('doc_4', 'contx_21'), ('doc_4', 'contx_22'), ('doc_4', 'contx_23'), ('doc_4', 'contx_24'), ('doc_5', 'contx_25'), ('doc_5', 'contx_26'), ('doc_5', 'contx_27'), ('doc_5', 'contx_28'), ('doc_5', 'contx_29'), ('doc_6', 'contx_30'), ('doc_6', 'contx_31'), ('doc_6', 'contx_32'), ('doc_6', 'contx_33'), ('doc_6', 'contx_34'), ('doc_7', 'contx_35'), ('doc_7', 'contx_36'), ('doc_7', 'contx_37'), ('doc_7', 'contx_38'), ('doc_7', 'contx_39'), ('doc_8', 'contx_40'), ('doc_8', 'contx_41'), ('doc_8', 'contx_42'), ('doc_8', 'contx_43'

#### Test graph Conectivity with networkx

In [ ]:
# Define Bipartide graph
B = nx.Graph()
B.add_nodes_from(document_nodes_test, bipartite=0)
B.add_nodes_from(context_nodes_test, bipartite=1)
B.add_edges_from(edges_test)

time: 2.94 s (started: 2024-10-16 20:53:43 +00:00)


In [ ]:
nx.is_connected(B)

False

time: 482 ms (started: 2024-10-16 20:53:46 +00:00)


In [ ]:
# Number of conected elements
# if == 1 -> all elements of the graph are conected
print(nx.number_connected_components(B))

10
time: 458 ms (started: 2024-10-16 20:53:46 +00:00)


In [ ]:
# size of a cluster
print(len(nx.node_connected_component(B,'doc_0')))

40140
time: 444 ms (started: 2024-10-16 20:53:47 +00:00)


### Creating Graph HeteroData Object

In [ ]:
# Defining nodes, edges and class labels
from torch_geometric.data import HeteroData

data = HeteroData()

# nodes
data['document'].x = document_nodes
data['concept'].x = context_nodes

# edges
data['document', 'has', 'concept'].edge_index = edges

#class labels
data['document'].y = y


time: 1.13 ms (started: 2024-10-16 20:53:47 +00:00)


In [ ]:
# Setting graph to undirected
data = T.ToUndirected()(data)

time: 18.8 ms (started: 2024-10-16 20:53:47 +00:00)


In [ ]:
# Remove duplicate edges
data = T.RemoveDuplicatedEdges()(data)

time: 501 ms (started: 2024-10-16 20:53:47 +00:00)


In [ ]:
# Ensure date in using gpu
data = data.to(device)

time: 26.1 ms (started: 2024-10-16 20:53:48 +00:00)


In [ ]:
data

HeteroData(
  document={
    x=[8817, 384],
    y=[8817],
  },
  concept={ x=[31332, 384] },
  (document, has, concept)={ edge_index=[2, 1098776] },
  (concept, rev_has, document)={ edge_index=[2, 1098776] }
)

time: 5.84 ms (started: 2024-10-16 20:53:48 +00:00)


## Creating all masks

In [ ]:
try:
    os.makedirs('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks_big/')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

time: 5.15 ms (started: 2024-10-16 20:53:48 +00:00)


### Rotulated = 20%

In [ ]:
rotulated_perc = 20
test_perc = (100 - rotulated_perc)/100
print(test_perc)

0.8
time: 493 µs (started: 2024-10-16 20:53:48 +00:00)


In [ ]:
# rotulated_perc = 80
# test_perc = (100 - rotulated_perc)/100
# print(test_perc)

# n_samples = len(y)
# print('all:',np.bincount(y))
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_perc, random_state=0)
# x = np.zeros(n_samples)
# for train_idx, test_idx in splitter.split(x, y):
#     train_mask = np.full(n_samples, False, dtype=bool)
#     val_mask = np.full(n_samples, False, dtype=bool)
#     test_mask = np.full(n_samples, False, dtype=bool)

#     train_mask[train_idx] = True
#     test_mask[test_idx] = True

#     print('train:',np.bincount(y[train_idx]))
#     print('test:',np.bincount(y[test_mask]))

#     print('train vs test:',np.bincount(train_mask))
#     print(np.bincount(val_mask))
#     print(np.bincount(test_mask))

#     train_mask = torch.tensor(train_mask)
#     val_mask = torch.tensor(val_mask)
#     test_mask = torch.tensor(test_mask)

time: 199 µs (started: 2024-10-16 20:53:48 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
for i in range(10):

    train_mask, val_mask, test_mask = creat_equal_masks(test_perc,y)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks_big/mask_rot'+str(rotulated_perc)+'%_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([train_mask, val_mask, test_mask], f)

all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [189 127  20 198 111  71 192  79 519  99 101  57]
test: [ 758  508   80  793  446  283  767  318 2076  396  404  225]
train vs test: [7054 1763]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [189 127  20 198 111  71 192  79 519  99 101  57]
test: [ 758  508   80  793  446  283  767  318 2076  396  404  225]
train vs test: [7054 1763]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [189 127  20 198 111  71 192  79 519  99 101  57]
test: [ 758  508   80  793  446  283  767  318 2076  396  404  225]
train vs test: [7054 1763]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [189 127  20 198 111  71 192  79 519  99 101  57]
test: [ 758  508   80  793  446  283  767  318 2076  396  404  225]
train vs test: [7054 1763]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [189 127  20 198 111  71 192  79 519  99 101  57]


### Rotulated = 40%

In [ ]:
rotulated_perc = 40
test_perc = (100 - rotulated_perc)/100
print(test_perc)

0.6
time: 1.5 ms (started: 2024-10-16 20:53:48 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
for i in range(10):

    train_mask, val_mask, test_mask = creat_equal_masks(test_perc,y)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks_big/mask_rot'+str(rotulated_perc)+'%_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([train_mask, val_mask, test_mask], f)

all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 379  254   40  396  223  141  383  159 1038  198  202  113]
test: [ 568  381   60  595  334  213  576  238 1557  297  303  169]
train vs test: [5291 3526]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 379  254   40  396  223  141  383  159 1038  198  202  113]
test: [ 568  381   60  595  334  213  576  238 1557  297  303  169]
train vs test: [5291 3526]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 379  254   40  396  223  141  383  159 1038  198  202  113]
test: [ 568  381   60  595  334  213  576  238 1557  297  303  169]
train vs test: [5291 3526]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 379  254   40  396  223  141  383  159 1038  198  202  113]
test: [ 568  381   60  595  334  213  576  238 1557  297  303  169]
train vs test: [5291 3526]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 

### Rotulated = 60%

In [ ]:
rotulated_perc = 60
test_perc = (100 - rotulated_perc)/100
print(test_perc)

0.4
time: 517 µs (started: 2024-10-16 20:53:48 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
for i in range(10):

    train_mask, val_mask, test_mask = creat_equal_masks(test_perc,y)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks_big/mask_rot'+str(rotulated_perc)+'%_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([train_mask, val_mask, test_mask], f)

all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 568  381   60  595  334  213  575  238 1557  297  303  169]
test: [ 379  254   40  396  223  141  384  159 1038  198  202  113]
train vs test: [3527 5290]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 568  381   60  595  334  213  575  238 1557  297  303  169]
test: [ 379  254   40  396  223  141  384  159 1038  198  202  113]
train vs test: [3527 5290]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 568  381   60  595  334  213  575  238 1557  297  303  169]
test: [ 379  254   40  396  223  141  384  159 1038  198  202  113]
train vs test: [3527 5290]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 568  381   60  595  334  213  575  238 1557  297  303  169]
test: [ 379  254   40  396  223  141  384  159 1038  198  202  113]
train vs test: [3527 5290]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 

### Rotulated = 80%

In [ ]:
rotulated_perc = 80
test_perc = (100 - rotulated_perc)/100
print(test_perc)

0.2
time: 1.67 ms (started: 2024-10-16 20:53:48 +00:00)


#### Creating mask Split using RandomNodeSplit (if split not done before)

In [ ]:
for i in range(10):

    train_mask, val_mask, test_mask = creat_equal_masks(test_perc,y)

    with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks_big/mask_rot'+str(rotulated_perc)+'%_'+str(i)+'.pkl', 'wb') as f:
        pickle.dump([train_mask, val_mask, test_mask], f)

all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 757  508   80  793  445  283  767  318 2076  396  404  226]
test: [190 127  20 198 112  71 192  79 519  99 101  56]
train vs test: [1764 7053]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 757  508   80  793  445  283  767  318 2076  396  404  226]
test: [190 127  20 198 112  71 192  79 519  99 101  56]
train vs test: [1764 7053]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 757  508   80  793  445  283  767  318 2076  396  404  226]
test: [190 127  20 198 112  71 192  79 519  99 101  56]
train vs test: [1764 7053]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 757  508   80  793  445  283  767  318 2076  396  404  226]
test: [190 127  20 198 112  71 192  79 519  99 101  56]
train vs test: [1764 7053]
all: [ 947  635  100  991  557  354  959  397 2595  495  505  282]
train: [ 757  508   80  793  445  283  767  318 2076  396

## TRAINING

### Training rotulated base = 20% ❎

In [ ]:
rotulated_perc = 20

time: 311 µs (started: 2024-10-16 20:53:49 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 80.4366, Train: 0.0369, Test: 0.0380
Early stopping:  nan
Epoch: 002, Loss: 266.7786, Train: 0.3279, Test: 0.3262
Early stopping:  131.76372082229022
Epoch: 003, Loss: 397.7393, Train: 0.1441, Test: 0.1452
Early stopping:  159.45484454691706
Epoch: 004, Loss: 346.3765, Train: 0.2762, Test: 0.2771
Early stopping:  139.12017091653956
Epoch: 005, Loss: 311.2749, Train: 0.0777, Test: 0.0764
Early stopping:  121.70199988461948
Epoch: 006, Loss: 485.4179, Train: 0.1214, Test: 0.1202
Early stopping:  84.25016260643493
Epoch: 007, Loss: 504.2057, Train: 0.0516, Test: 0.0466
Early stopping:  84.41346014455154
Epoch: 008, Loss: 585.7357, Train: 0.2036, Test: 0.2023
Early stopping:  114.6097868179186
Epoch: 009, Loss: 440.5719, Train: 0.3029, Test: 0.2981
Early stopping:  100.94334158627193
Epoch: 010, Loss: 459.1966, Train: 0.3006, Test: 0.2927
Early stopping:  56.24458594790368
Epoch: 011, Loss: 385.2913, Train: 0.1242, Test: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 82.1807, Train: 0.2944, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 508.6463, Train: 0.0556, Test: 0.0564
Early stopping:  301.55670624855753
Epoch: 003, Loss: 597.6083, Train: 0.1072, Test: 0.1075
Early stopping:  275.51548524318895
Epoch: 004, Loss: 427.7386, Train: 0.1151, Test: 0.1128
Early stopping:  225.5114025421205
Epoch: 005, Loss: 345.9026, Train: 0.0482, Test: 0.0491
Early stopping:  197.02187147100375
Epoch: 006, Loss: 301.2816, Train: 0.1282, Test: 0.1147
Early stopping:  120.134596794617
Epoch: 007, Loss: 235.1357, Train: 0.2944, Test: 0.2943
Early stopping:  139.61149358935188
Epoch: 008, Loss: 258.5432, Train: 0.2989, Test: 0.2976
Early stopping:  76.50270284422035
Epoch: 009, Loss: 255.6884, Train: 0.2745, Test: 0.2786
Early stopping:  44.322495811203275
Epoch: 010, Loss: 226.2138, Train: 0.3052, Test: 0.3027
Early stopping:  29.058392130917976
Epoch: 011, Loss: 180.3377, Train: 0.3012, Test: 0.2875
Early stopping:  31.522413239893464
Epoch: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 74.1266, Train: 0.1299, Test: 0.1277
Early stopping:  nan
Epoch: 002, Loss: 411.8520, Train: 0.1463, Test: 0.1453
Early stopping:  238.807960285064
Epoch: 003, Loss: 421.8684, Train: 0.2938, Test: 0.2932
Early stopping:  197.94073271654287
Epoch: 004, Loss: 486.2402, Train: 0.0805, Test: 0.0751
Early stopping:  185.87588835404284
Epoch: 005, Loss: 382.6359, Train: 0.0925, Test: 0.0910
Early stopping:  161.69458498224856
Epoch: 006, Loss: 419.5711, Train: 0.1214, Test: 0.1219
Early stopping:  37.932905150866205
Epoch: 007, Loss: 460.1505, Train: 0.0635, Test: 0.0634
Early stopping:  40.02025196889204
Epoch: 008, Loss: 462.5825, Train: 0.2235, Test: 0.2219
Early stopping:  41.040065788606405
Epoch: 009, Loss: 296.1383, Train: 0.2961, Test: 0.2959
Early stopping:  68.77061625868883
Epoch: 010, Loss: 289.9023, Train: 0.0998, Test: 0.1019
Early stopping:  86.31273359449601
Epoch: 011, Loss: 250.9393, Train: 0.1083, Test: 0.1107
Early stopping:  101.38409331079794
Epoch: 01

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 91.9239, Train: 0.0908, Test: 0.0910
Early stopping:  nan
Epoch: 002, Loss: 367.1638, Train: 0.2927, Test: 0.2892
Early stopping:  194.62404736405585
Epoch: 003, Loss: 439.1105, Train: 0.2791, Test: 0.2756
Early stopping:  183.2447986245262
Epoch: 004, Loss: 262.3686, Train: 0.0533, Test: 0.0512
Early stopping:  150.76004581789775
Epoch: 005, Loss: 346.0779, Train: 0.1514, Test: 0.1464
Early stopping:  132.9368893879375
Epoch: 006, Loss: 371.0906, Train: 0.0942, Test: 0.0868
Early stopping:  63.47884115622842
Epoch: 007, Loss: 421.9782, Train: 0.1140, Test: 0.1177
Early stopping:  70.03285937913292
Epoch: 008, Loss: 407.5714, Train: 0.1055, Test: 0.1094
Early stopping:  63.120027349350856
Epoch: 009, Loss: 351.6394, Train: 0.1821, Test: 0.1870
Early stopping:  33.74989671989098
Epoch: 010, Loss: 312.5235, Train: 0.1696, Test: 0.1663
Early stopping:  43.90720542730857
Epoch: 011, Loss: 243.2550, Train: 0.1497, Test: 0.1572
Early stopping:  72.90926720638436
Epoch: 012,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 62.7986, Train: 0.2303, Test: 0.2413
Early stopping:  nan
Epoch: 002, Loss: 161.7175, Train: 0.2938, Test: 0.2947
Early stopping:  69.94626511048492
Epoch: 003, Loss: 337.2598, Train: 0.0113, Test: 0.0118
Early stopping:  139.0018049741378
Epoch: 004, Loss: 594.2366, Train: 0.1202, Test: 0.1259
Early stopping:  232.99948931733732
Epoch: 005, Loss: 485.2892, Train: 0.1254, Test: 0.1225
Early stopping:  220.0504840643963
Epoch: 006, Loss: 426.2434, Train: 0.0630, Test: 0.0604
Early stopping:  163.09596271689153
Epoch: 007, Loss: 378.4779, Train: 0.0732, Test: 0.0716
Early stopping:  100.34600859501246
Epoch: 008, Loss: 290.9341, Train: 0.2734, Test: 0.2716
Early stopping:  113.92070377179814
Epoch: 009, Loss: 194.2830, Train: 0.2751, Test: 0.2641
Early stopping:  114.60421845416448
Epoch: 010, Loss: 165.4197, Train: 0.1537, Test: 0.1520
Early stopping:  112.99086421008762
Epoch: 011, Loss: 139.7855, Train: 0.1730, Test: 0.1667
Early stopping:  99.08603457202418
Epoch: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 436 ms (started: 2024-10-16 20:55:50 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4706, Train: 0.2944, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 2.2602, Train: 0.2944, Test: 0.2943
Early stopping:  0.14879472822067816
Epoch: 003, Loss: 2.1703, Train: 0.2944, Test: 0.2943
Early stopping:  0.15414674724688404
Epoch: 004, Loss: 2.1238, Train: 0.3006, Test: 0.2994
Early stopping:  0.1537369243800661
Epoch: 005, Loss: 2.0568, Train: 0.3579, Test: 0.3459
Early stopping:  0.16023597728178407
Epoch: 006, Loss: 2.0030, Train: 0.3897, Test: 0.3740
Early stopping:  0.09979407067052186
Epoch: 007, Loss: 1.9466, Train: 0.4056, Test: 0.3857
Early stopping:  0.08991586113184191
Epoch: 008, Loss: 1.8789, Train: 0.4078, Test: 0.3889
Early stopping:  0.09492743508595801
Epoch: 009, Loss: 1.8109, Train: 0.4180, Test: 0.3981
Early stopping:  0.0975186068299187
Epoch: 010, Loss: 1.7480, Train: 0.4413, Test: 0.4256
Early stopping:  0.10213186894071245
Epoch: 011, Loss: 1.6834, Train: 0.4651, Test: 0.4426
Earl

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 407 ms (started: 2024-10-16 20:57:57 +00:00)


### Training rotulated base = 40% ❎

In [ ]:
rotulated_perc = 40

time: 574 µs (started: 2024-10-16 20:57:57 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 85.5463, Train: 0.2969, Test: 0.2952
Early stopping:  nan
Epoch: 002, Loss: 350.4079, Train: 0.0752, Test: 0.0741
Early stopping:  187.2854461935339
Epoch: 003, Loss: 644.9293, Train: 0.1387, Test: 0.1438
Early stopping:  279.82254272530844
Epoch: 004, Loss: 623.5221, Train: 0.0771, Test: 0.0858
Early stopping:  263.6714406960082
Epoch: 005, Loss: 509.4789, Train: 0.0559, Test: 0.0580
Early stopping:  231.37055440002882
Epoch: 006, Loss: 401.6621, Train: 0.0434, Test: 0.0474
Early stopping:  130.59498414425684
Epoch: 007, Loss: 272.4460, Train: 0.0805, Test: 0.0898
Early stopping:  155.99932765316026
Epoch: 008, Loss: 165.8453, Train: 0.1293, Test: 0.1291
Early stopping:  182.2790868372921
Epoch: 009, Loss: 89.4267, Train: 0.1254, Test: 0.1187
Early stopping:  170.70937064439053
Epoch: 010, Loss: 47.6080, Train: 0.2725, Test: 0.2716
Early stopping:  143.5500315950319
Epoch: 011, Loss: 37.8439, Train: 0.2782, Test: 0.28

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 151.2359, Train: 0.0709, Test: 0.0711
Early stopping:  nan
Epoch: 002, Loss: 334.6982, Train: 0.1117, Test: 0.1121
Early stopping:  129.72747642126072
Epoch: 003, Loss: 388.5026, Train: 0.3134, Test: 0.3139
Early stopping:  124.39776102464516
Epoch: 004, Loss: 434.2994, Train: 0.1242, Test: 0.1170
Early stopping:  124.16102399509967
Epoch: 005, Loss: 586.6841, Train: 0.0726, Test: 0.0663
Early stopping:  158.2088246102898
Epoch: 006, Loss: 615.4060, Train: 0.1245, Test: 0.1206
Early stopping:  123.45305876271122
Epoch: 007, Loss: 631.4198, Train: 0.1103, Test: 0.1077
Early stopping:  111.764723299568
Epoch: 008, Loss: 636.4448, Train: 0.2947, Test: 0.2939
Early stopping:  84.19222042823868
Epoch: 009, Loss: 655.3168, Train: 0.2853, Test: 0.2833
Early stopping:  25.746082645103353
Epoch: 010, Loss: 601.1988, Train: 0.1194, Test: 0.1196
Early stopping:  20.652625782894997
Epoch: 011, Loss: 521.0013, Train: 0.2984, Test: 0.2943
Early stopping:  52.92853441515662
Epoch: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 62.8202, Train: 0.0570, Test: 0.0578
Early stopping:  nan
Epoch: 002, Loss: 613.8536, Train: 0.2941, Test: 0.2948
Early stopping:  389.639414392369
Epoch: 003, Loss: 568.1838, Train: 0.0587, Test: 0.0550
Early stopping:  305.80925132713764
Epoch: 004, Loss: 518.6346, Train: 0.0533, Test: 0.0542
Early stopping:  255.017040357249
Epoch: 005, Loss: 637.6130, Train: 0.1622, Test: 0.1588
Early stopping:  237.73218814700758
Epoch: 006, Loss: 536.9227, Train: 0.1220, Test: 0.1191
Early stopping:  50.257806795254226
Epoch: 007, Loss: 494.7418, Train: 0.0644, Test: 0.0626
Early stopping:  55.24387130972411
Epoch: 008, Loss: 443.4964, Train: 0.1767, Test: 0.1669
Early stopping:  71.43974578475317
Epoch: 009, Loss: 352.4517, Train: 0.2981, Test: 0.2956
Early stopping:  106.12444185233323
Epoch: 010, Loss: 350.9095, Train: 0.2969, Test: 0.2954
Early stopping:  83.5350960527516
Epoch: 011, Loss: 308.9616, Train: 0.1151, Test: 0.1119
Early stopping:  76.38357227827606
Epoch: 012, L

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 73.8669, Train: 0.1157, Test: 0.1185
Early stopping:  nan
Epoch: 002, Loss: 432.9494, Train: 0.2969, Test: 0.2984
Early stopping:  253.90969535569099
Epoch: 003, Loss: 462.3354, Train: 0.1174, Test: 0.1230
Early stopping:  216.29900962157166
Epoch: 004, Loss: 574.2357, Train: 0.0533, Test: 0.0535
Early stopping:  216.71100799148374
Epoch: 005, Loss: 681.4073, Train: 0.0899, Test: 0.0820
Early stopping:  229.55160843611253
Epoch: 006, Loss: 567.4221, Train: 0.0678, Test: 0.0648
Early stopping:  99.177042713777
Epoch: 007, Loss: 405.4456, Train: 0.0993, Test: 0.0949
Early stopping:  107.28466953486291
Epoch: 008, Loss: 358.6193, Train: 0.2686, Test: 0.2480
Early stopping:  132.642381156352
Epoch: 009, Loss: 244.4176, Train: 0.3054, Test: 0.3009
Early stopping:  173.15183618337687
Epoch: 010, Loss: 210.4128, Train: 0.3057, Test: 0.2977
Early stopping:  142.11729273022587
Epoch: 011, Loss: 152.4014, Train: 0.3037, Test: 0.2916
Early stopping:  105.05081540830564
Epoch: 01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 74.1299, Train: 0.0621, Test: 0.0588
Early stopping:  nan
Epoch: 002, Loss: 282.7837, Train: 0.2862, Test: 0.2894
Early stopping:  147.54050679265072
Epoch: 003, Loss: 343.5704, Train: 0.1123, Test: 0.1115
Early stopping:  141.32092852281497
Epoch: 004, Loss: 496.9581, Train: 0.0176, Test: 0.0168
Early stopping:  175.12180597247902
Epoch: 005, Loss: 540.8431, Train: 0.0335, Test: 0.0346
Early stopping:  186.18136892121785
Epoch: 006, Loss: 463.8925, Train: 0.0479, Test: 0.0495
Early stopping:  108.35660635053131
Epoch: 007, Loss: 370.5264, Train: 0.2116, Test: 0.2053
Early stopping:  83.75526825751993
Epoch: 008, Loss: 291.0767, Train: 0.3227, Test: 0.3188
Early stopping:  100.89561157887259
Epoch: 009, Loss: 279.4119, Train: 0.1412, Test: 0.1366
Early stopping:  112.47765800222965
Epoch: 010, Loss: 266.8419, Train: 0.1617, Test: 0.1487
Early stopping:  82.97305607386389
Epoch: 011, Loss: 291.8959, Train: 0.1999, Test: 0.1907
Early stopping:  40.751059784643466
Epoch:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 422 ms (started: 2024-10-16 20:59:39 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4703, Train: 0.2944, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 2.2619, Train: 0.2944, Test: 0.2943
Early stopping:  0.14740236493327227
Epoch: 003, Loss: 2.1668, Train: 0.2944, Test: 0.2943
Early stopping:  0.15526623986188318
Epoch: 004, Loss: 2.1286, Train: 0.2958, Test: 0.2975
Early stopping:  0.15293551952546114
Epoch: 005, Loss: 2.0588, Train: 0.3395, Test: 0.3404
Early stopping:  0.15933485305012965
Epoch: 006, Loss: 1.9953, Train: 0.3888, Test: 0.3780
Early stopping:  0.10204820203415521
Epoch: 007, Loss: 1.9357, Train: 0.3999, Test: 0.3901
Early stopping:  0.09446956401599091
Epoch: 008, Loss: 1.8697, Train: 0.4027, Test: 0.3895
Early stopping:  0.1013502569978026
Epoch: 009, Loss: 1.8048, Train: 0.4121, Test: 0.4001
Early stopping:  0.10018632036780564
Epoch: 010, Loss: 1.7447, Train: 0.4353, Test: 0.4213
Early stopping:  0.09996677811908394
Epoch: 011, Loss: 1.6803, Train: 0.4702, Test: 0.4489
Ear

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 398 ms (started: 2024-10-16 21:02:10 +00:00)


### Training rotulated base = 60% ❎

In [ ]:
rotulated_perc = 60

time: 426 µs (started: 2024-10-16 21:02:11 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 40.4308, Train: 0.1287, Test: 0.1222
Early stopping:  nan
Epoch: 002, Loss: 469.9228, Train: 0.1095, Test: 0.1094
Early stopping:  303.6967492337501
Epoch: 003, Loss: 519.7562, Train: 0.0654, Test: 0.0649
Early stopping:  263.5335643106124
Epoch: 004, Loss: 248.8673, Train: 0.1053, Test: 0.1004
Early stopping:  220.30126356761153
Epoch: 005, Loss: 131.3970, Train: 0.1098, Test: 0.1038
Early stopping:  208.55315660349794
Epoch: 006, Loss: 61.9364, Train: 0.2032, Test: 0.2039
Early stopping:  202.4563137276681
Epoch: 007, Loss: 62.1564, Train: 0.2520, Test: 0.2450
Early stopping:  191.88831146213124
Epoch: 008, Loss: 36.2407, Train: 0.2253, Test: 0.2112
Early stopping:  86.260985613348
Epoch: 009, Loss: 11.3510, Train: 0.1172, Test: 0.1123
Early stopping:  44.81984629520991
Epoch: 010, Loss: 6.1310, Train: 0.1399, Test: 0.1335
Early stopping:  26.719333031669997
Epoch: 011, Loss: 4.4831, Train: 0.1974, Test: 0.1908
Early

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 100.1794, Train: 0.2879, Test: 0.2883
Early stopping:  nan
Epoch: 002, Loss: 341.1819, Train: 0.0934, Test: 0.0961
Early stopping:  170.41446060087296
Epoch: 003, Loss: 376.4346, Train: 0.1172, Test: 0.1157
Early stopping:  150.3561459612515
Epoch: 004, Loss: 275.3536, Train: 0.0762, Test: 0.0731
Early stopping:  122.77300672833198
Epoch: 005, Loss: 197.2526, Train: 0.0575, Test: 0.0573
Early stopping:  111.62958563380847
Epoch: 006, Loss: 190.2493, Train: 0.0588, Test: 0.0598
Early stopping:  83.50220968389088
Epoch: 007, Loss: 126.5772, Train: 0.0775, Test: 0.0777
Early stopping:  95.91461524090852
Epoch: 008, Loss: 70.9712, Train: 0.2125, Test: 0.2092
Early stopping:  77.33483599352822
Epoch: 009, Loss: 56.6070, Train: 0.3019, Test: 0.2969
Early stopping:  65.23348113820043
Epoch: 010, Loss: 57.5756, Train: 0.2919, Test: 0.2866
Early stopping:  57.82724710433973
Epoch: 011, Loss: 28.7462, Train: 0.2662, Test: 0.2589
Early stopping:  36.122715907389065
Epoch: 012, L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 83.8132, Train: 0.2943, Test: 0.2900
Early stopping:  nan
Epoch: 002, Loss: 361.1487, Train: 0.1146, Test: 0.1154
Early stopping:  196.1058199643684
Epoch: 003, Loss: 568.0828, Train: 0.0635, Test: 0.0629
Early stopping:  242.9861777201259
Epoch: 004, Loss: 751.5924, Train: 0.0743, Test: 0.0740
Early stopping:  286.6915925309604
Epoch: 005, Loss: 714.3441, Train: 0.0696, Test: 0.0644
Early stopping:  276.71292674048874
Epoch: 006, Loss: 460.6747, Train: 0.1274, Test: 0.1324
Early stopping:  165.36229929302257
Epoch: 007, Loss: 347.0760, Train: 0.1180, Test: 0.1177
Early stopping:  169.8884256175462
Epoch: 008, Loss: 201.2636, Train: 0.1193, Test: 0.1145
Early stopping:  236.26909344170676
Epoch: 009, Loss: 117.1193, Train: 0.2658, Test: 0.2591
Early stopping:  234.3729683839389
Epoch: 010, Loss: 91.0065, Train: 0.2204, Test: 0.2203
Early stopping:  157.24427658811865
Epoch: 011, Loss: 72.1464, Train: 0.2149, Test: 0.2081
Early stopping:  112.75357081885932
Epoch: 012,

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 66.9890, Train: 0.2930, Test: 0.2926
Early stopping:  nan
Epoch: 002, Loss: 386.9687, Train: 0.0461, Test: 0.0462
Early stopping:  226.25983600510403
Epoch: 003, Loss: 531.6615, Train: 0.0972, Test: 0.0904
Early stopping:  237.78266237022518
Epoch: 004, Loss: 512.0292, Train: 0.1401, Test: 0.1341
Early stopping:  214.7343213857863
Epoch: 005, Loss: 381.9022, Train: 0.0688, Test: 0.0649
Early stopping:  185.99554278668478
Epoch: 006, Loss: 352.5399, Train: 0.0964, Test: 0.0879
Early stopping:  82.43695463808655
Epoch: 007, Loss: 302.7993, Train: 0.1189, Test: 0.1168
Early stopping:  100.75126035675834
Epoch: 008, Loss: 268.7824, Train: 0.0510, Test: 0.0499
Early stopping:  93.77583426449996
Epoch: 009, Loss: 212.3766, Train: 0.1393, Test: 0.1403
Early stopping:  67.19547978537976
Epoch: 010, Loss: 130.7694, Train: 0.3132, Test: 0.3136
Early stopping:  85.47924031891682
Epoch: 011, Loss: 128.4002, Train: 0.3236, Test: 0.3181
Early stopping:  79.05585653452563
Epoch: 012

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 80.4751, Train: 0.1121, Test: 0.1117
Early stopping:  nan
Epoch: 002, Loss: 263.5161, Train: 0.1163, Test: 0.1154
Early stopping:  129.42951101493065
Epoch: 003, Loss: 324.1829, Train: 0.2066, Test: 0.2141
Early stopping:  126.87128640124548
Epoch: 004, Loss: 342.9884, Train: 0.2949, Test: 0.2926
Early stopping:  119.77781045734537
Epoch: 005, Loss: 386.2307, Train: 0.1074, Test: 0.1038
Early stopping:  119.67160914210012
Epoch: 006, Loss: 497.4266, Train: 0.0410, Test: 0.0417
Early stopping:  87.19858483853321
Epoch: 007, Loss: 535.9965, Train: 0.0185, Test: 0.0190
Early stopping:  94.42911144560938
Epoch: 008, Loss: 518.7327, Train: 0.0730, Test: 0.0692
Early stopping:  86.15392108605006
Epoch: 009, Loss: 419.5856, Train: 0.1000, Test: 0.0978
Early stopping:  65.24727343747078
Epoch: 010, Loss: 343.5058, Train: 0.2994, Test: 0.2940
Early stopping:  80.28616087949459
Epoch: 011, Loss: 315.1766, Train: 0.2998, Test: 0.2997
Early stopping:  99.78106979577255
Epoch: 012

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 417 ms (started: 2024-10-16 21:03:42 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4839, Train: 0.2941, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 2.2826, Train: 0.2941, Test: 0.2937
Early stopping:  0.14234845203463395
Epoch: 003, Loss: 2.1696, Train: 0.2947, Test: 0.2943
Early stopping:  0.15924659944152372
Epoch: 004, Loss: 2.1468, Train: 0.3034, Test: 0.3022
Early stopping:  0.15404092501684624
Epoch: 005, Loss: 2.0768, Train: 0.3406, Test: 0.3377
Early stopping:  0.15911985379341978
Epoch: 006, Loss: 2.0139, Train: 0.3675, Test: 0.3632
Early stopping:  0.10142767049267519
Epoch: 007, Loss: 1.9600, Train: 0.3733, Test: 0.3680
Early stopping:  0.08811231978173045
Epoch: 008, Loss: 1.9028, Train: 0.3749, Test: 0.3689
Early stopping:  0.09578522282733015
Epoch: 009, Loss: 1.8453, Train: 0.3856, Test: 0.3828
Early stopping:  0.0907882696213658
Epoch: 010, Loss: 1.7892, Train: 0.4142, Test: 0.4086
Early stopping:  0.08916965662252974
Epoch: 011, Loss: 1.7295, Train: 0.4488, Test: 0.4409
Ear

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 437 ms (started: 2024-10-16 21:06:11 +00:00)


### Training rotulated base = 80% ❎

In [ ]:
rotulated_perc = 80

time: 1.89 ms (started: 2024-10-16 21:06:11 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 75.6125, Train: 0.1636, Test: 0.1548
Early stopping:  nan
Epoch: 002, Loss: 776.0373, Train: 0.1133, Test: 0.1122
Early stopping:  495.2750859474215
Epoch: 003, Loss: 713.9493, Train: 0.0577, Test: 0.0573
Early stopping:  387.7120055042322
Epoch: 004, Loss: 768.6353, Train: 0.0563, Test: 0.0499
Early stopping:  339.7608833995177
Epoch: 005, Loss: 415.7242, Train: 0.0503, Test: 0.0459
Early stopping:  303.6639185986307
Epoch: 006, Loss: 241.8154, Train: 0.0818, Test: 0.0737
Early stopping:  241.48472447705026
Epoch: 007, Loss: 114.1308, Train: 0.0654, Test: 0.0646
Early stopping:  286.582395310687
Epoch: 008, Loss: 35.6068, Train: 0.0715, Test: 0.0692
Early stopping:  291.41686191837067
Epoch: 009, Loss: 10.1635, Train: 0.0968, Test: 0.0907
Early stopping:  167.37412225682058
Epoch: 010, Loss: 3.5755, Train: 0.1411, Test: 0.1264
Early stopping:  100.04014325946417
Epoch: 011, Loss: 2.6748, Train: 0.1255, Test: 0.1139
Ea

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 57.6118, Train: 0.0320, Test: 0.0317
Early stopping:  nan
Epoch: 002, Loss: 778.1760, Train: 0.0727, Test: 0.0731
Early stopping:  509.5158735193501
Epoch: 003, Loss: 605.1738, Train: 0.1224, Test: 0.1247
Early stopping:  376.15750734254925
Epoch: 004, Loss: 378.1602, Train: 0.0832, Test: 0.0839
Early stopping:  311.35001418909434
Epoch: 005, Loss: 361.2312, Train: 0.1331, Test: 0.1264
Early stopping:  272.86336709292243
Epoch: 006, Loss: 280.0626, Train: 0.1076, Test: 0.1066
Early stopping:  205.56520059552204
Epoch: 007, Loss: 166.7249, Train: 0.1002, Test: 0.1037
Early stopping:  161.37830123264465
Epoch: 008, Loss: 142.6786, Train: 0.1484, Test: 0.1468
Early stopping:  108.29370320720331
Epoch: 009, Loss: 119.5859, Train: 0.2994, Test: 0.2971
Early stopping:  102.80683124890359
Epoch: 010, Loss: 139.4163, Train: 0.2990, Test: 0.3010
Early stopping:  63.92854601423803
Epoch: 011, Loss: 129.0179, Train: 0.3026, Test: 0.2993
Early stopping:  17.732388205317854
Epoch:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 87.4226, Train: 0.2819, Test: 0.2789
Early stopping:  nan
Epoch: 002, Loss: 338.4048, Train: 0.1116, Test: 0.1077
Early stopping:  177.47124838611336
Epoch: 003, Loss: 776.4841, Train: 0.1076, Test: 0.1083
Early stopping:  348.7385057756239
Epoch: 004, Loss: 1009.3166, Train: 0.0727, Test: 0.0731
Early stopping:  416.72667370631075
Epoch: 005, Loss: 868.3992, Train: 0.0493, Test: 0.0442
Early stopping:  387.4956785538777
Epoch: 006, Loss: 585.4974, Train: 0.0595, Test: 0.0539
Early stopping:  261.02049203647243
Epoch: 007, Loss: 592.6985, Train: 0.0689, Test: 0.0618
Early stopping:  181.9422522436258
Epoch: 008, Loss: 499.3116, Train: 0.0842, Test: 0.0805
Early stopping:  216.98237569808168
Epoch: 009, Loss: 325.5820, Train: 0.0581, Test: 0.0533
Early stopping:  196.49561590910335
Epoch: 010, Loss: 252.9663, Train: 0.0817, Test: 0.0737
Early stopping:  154.47394571526783
Epoch: 011, Loss: 162.4672, Train: 0.3028, Test: 0.2942
Early stopping:  176.77540815264962
Epoch:

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 87.5115, Train: 0.0601, Test: 0.0595
Early stopping:  nan
Epoch: 002, Loss: 399.6353, Train: 0.0424, Test: 0.0408
Early stopping:  220.70486788437668
Epoch: 003, Loss: 497.4770, Train: 0.1153, Test: 0.1145
Early stopping:  214.11291318954326
Epoch: 004, Loss: 514.6811, Train: 0.1072, Test: 0.1088
Early stopping:  198.13115253505092
Epoch: 005, Loss: 400.1791, Train: 0.2968, Test: 0.2948
Early stopping:  171.9608066665553
Epoch: 006, Loss: 391.6611, Train: 0.2956, Test: 0.2942
Early stopping:  60.06225602421492
Epoch: 007, Loss: 368.5255, Train: 0.1231, Test: 0.1224
Early stopping:  66.63492106714443
Epoch: 008, Loss: 337.8420, Train: 0.1935, Test: 0.1865
Early stopping:  67.16092888001339
Epoch: 009, Loss: 274.0680, Train: 0.2783, Test: 0.2710
Early stopping:  51.017228961081216
Epoch: 010, Loss: 218.3507, Train: 0.1839, Test: 0.1684
Early stopping:  71.12041650432678
Epoch: 011, Loss: 184.7065, Train: 0.2583, Test: 0.2409
Early stopping:  77.53167873349109
Epoch: 012

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 50.1527, Train: 0.1137, Test: 0.1173
Early stopping:  nan
Epoch: 002, Loss: 416.4834, Train: 0.2942, Test: 0.2937
Early stopping:  259.03493016290383
Epoch: 003, Loss: 503.5924, Train: 0.0770, Test: 0.0816
Early stopping:  240.6220235536702
Epoch: 004, Loss: 510.2979, Train: 0.0810, Test: 0.0782
Early stopping:  217.55717074998188
Epoch: 005, Loss: 560.9454, Train: 0.1348, Test: 0.1327
Early stopping:  206.83404367566308
Epoch: 006, Loss: 604.5966, Train: 0.1082, Test: 0.1122
Early stopping:  70.54925891285508
Epoch: 007, Loss: 561.4997, Train: 0.0702, Test: 0.0652
Early stopping:  41.672248225850275
Epoch: 008, Loss: 524.5591, Train: 0.0688, Test: 0.0720
Early stopping:  36.83750831575918
Epoch: 009, Loss: 468.1175, Train: 0.1243, Test: 0.1423
Early stopping:  50.99366078021521
Epoch: 010, Loss: 477.0154, Train: 0.2938, Test: 0.2937
Early stopping:  57.40839203569291
Epoch: 011, Loss: 505.4342, Train: 0.2935, Test: 0.2925
Early stopping:  37.7206385769013
Epoch: 012,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 488 ms (started: 2024-10-16 21:07:38 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.5130, Train: 0.2941, Test: 0.2931
Early stopping:  nan
Epoch: 002, Loss: 2.3115, Train: 0.2941, Test: 0.2931
Early stopping:  0.14250439537412424
Epoch: 003, Loss: 2.1918, Train: 0.2943, Test: 0.2942
Early stopping:  0.16234692044629728
Epoch: 004, Loss: 2.1541, Train: 0.3004, Test: 0.2993
Early stopping:  0.16154336358839
Epoch: 005, Loss: 2.0995, Train: 0.3391, Test: 0.3362
Early stopping:  0.16440392452614266
Epoch: 006, Loss: 2.0321, Train: 0.3754, Test: 0.3702
Early stopping:  0.10486620594515077
Epoch: 007, Loss: 1.9733, Train: 0.3910, Test: 0.3815
Early stopping:  0.0887950502427705
Epoch: 008, Loss: 1.9230, Train: 0.3866, Test: 0.3815
Early stopping:  0.09312488687286205
Epoch: 009, Loss: 1.8753, Train: 0.3892, Test: 0.3770
Early stopping:  0.0883672000006449
Epoch: 010, Loss: 1.8276, Train: 0.4029, Test: 0.3929
Early stopping:  0.08021449561659852
Epoch: 011, Loss: 1.7754, Train: 0.4310, Test: 0.4110
Early s

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 437 ms (started: 2024-10-16 21:10:06 +00:00)


--------------------------------------
# Hetrogenous Graph for Keyphrase = 3
--------------------------------------

In [ ]:
print(df["class"].unique())
class_number = len(df["class"].unique())
print(class_number)

['basic_materials' 'capital_goods' 'conglomerates_industry'
 'consumer_cyclical' 'consumer_non-cyclical' 'energy' 'financial'
 'healthcare' 'services' 'technology' 'transportation' 'utilities']
12
time: 2.66 ms (started: 2024-10-16 21:10:07 +00:00)


In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase3"

model_name = dataset_name+"_"+keyphrase

time: 468 µs (started: 2024-10-16 21:10:07 +00:00)


## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

document_nodes = []
context_nodes = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

time: 17.1 s (started: 2024-10-16 21:10:07 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 8817
number of context nodes: 35984
number of shared contexts: 6890
number of direct edges (first dimension): 42874
number of direct edges (second dimension): 42874
time: 6.57 ms (started: 2024-10-16 21:10:24 +00:00)


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

time: 2min 7s (started: 2024-10-16 21:10:24 +00:00)


##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

time: 197 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

time: 58.1 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# Show documents nodes
document_nodes

tensor([[-5.8602e-02,  1.4557e-02,  1.7552e-02,  ..., -1.8911e-02,
         -1.8681e-02, -2.2412e-02],
        [-1.2433e-01, -1.2993e-02,  1.4001e-02,  ..., -1.7944e-02,
          1.1914e-02, -8.1062e-03],
        [-8.5865e-02,  4.3530e-02, -7.1712e-02,  ...,  5.6326e-02,
         -4.5610e-02,  3.1375e-02],
        ...,
        [-1.6584e-02, -3.9770e-02, -2.2835e-02,  ..., -4.2101e-05,
          1.2257e-02,  2.9048e-02],
        [ 3.1864e-02, -1.0648e-02, -3.8576e-02,  ..., -4.5610e-03,
         -1.7202e-02,  4.2433e-02],
        [ 4.3771e-02,  3.8128e-02,  2.0526e-02,  ..., -1.0110e-02,
          3.8847e-02,  4.4935e-02]])

time: 5.14 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# Quantity of document nodes
len(document_nodes)

8817

time: 2.71 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# Show context nodes
context_nodes

tensor([[-0.0714,  0.0858, -0.0434,  ...,  0.0149,  0.0713,  0.0182],
        [-0.0430,  0.0382,  0.0263,  ...,  0.0298,  0.0570, -0.0218],
        [-0.0816,  0.0714, -0.0450,  ...,  0.0140,  0.0811,  0.0313],
        ...,
        [-0.0464,  0.0437,  0.0029,  ...,  0.0401,  0.0498,  0.0721],
        [-0.0457, -0.0080, -0.0750,  ...,  0.0681, -0.0170,  0.0555],
        [-0.0652,  0.0149, -0.0086,  ...,  0.0067, -0.0111,  0.0143]])

time: 4 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# Quantity of context nodes
len(context_nodes)

35984

time: 2.82 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

1946479
1946479
time: 481 µs (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  8816,  8816,  8816],
        [    0,     1,     2,  ..., 32218, 12105, 23313]])
time: 1.85 ms (started: 2024-10-16 21:12:31 +00:00)


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['basic_materials' 'capital_goods' 'conglomerates_industry'
 'consumer_cyclical' 'consumer_non-cyclical' 'energy' 'financial'
 'healthcare' 'services' 'technology' 'transportation' 'utilities']
time: 1.68 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'basic_materials': 0, 'capital_goods': 1, 'conglomerates_industry': 2, 'consumer_cyclical': 3, 'consumer_non-cyclical': 4, 'energy': 5, 'financial': 6, 'healthcare': 7, 'services': 8, 'technology': 9, 'transportation': 10, 'utilities': 11} 



<ipython-input-281-a018e46c97dd>:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  labels["class"][i] = dit[labels["class"][i]]
<ipython-input-281-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,class
0,0
1,0
2,0
3,0
4,0


time: 841 ms (started: 2024-10-16 21:12:31 +00:00)


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([ 0,  0,  0,  ..., 11, 11, 11])

time: 5.26 ms (started: 2024-10-16 21:12:32 +00:00)


In [ ]:
len(y)

8817

time: 3.17 ms (started: 2024-10-16 21:12:32 +00:00)


### Testing Graph with Networkx




#### Defining overal graph in networkx maner

In [ ]:
# Run graph Representation for networkx
all_contexts_list =[]
edges_test = []
cont = 0
new_edge_cont = 0

document_nodes_test = []
context_nodes_test = []

# Creating graph nodes and edges
for i in range(len(df)):
    # adding new documents for every node
    document_nodes_test.append("doc_"+str(i)) # in the actual graph nodes -> documents embeddings

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list:
            # add a new context node for every new keyphrase in df
            context_nodes_test.append("contx_"+str(new_edge_cont)) # in the actual graph nodes -> context embeddings

            # add a new edge between doc and new context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(new_edge_cont)))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(all_contexts_list.index(key[0]))))
            cont+=1

time: 13.6 s (started: 2024-10-16 21:12:32 +00:00)


In [ ]:
edges_test = [("doc_"+str(i[0]),"contx_"+str(i[1])) for i in edges_tuple]

time: 1.17 s (started: 2024-10-16 21:12:46 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes_test))
print("number of context nodes:",len(context_nodes_test))
print("number of edges:",len(edges_test))

number of document nodes: 8817
number of context nodes: 35984
number of edges: 1946479
time: 2.71 ms (started: 2024-10-16 21:12:47 +00:00)


In [ ]:
print(edges_test[0:100])

[('doc_0', 'contx_0'), ('doc_0', 'contx_1'), ('doc_0', 'contx_2'), ('doc_0', 'contx_3'), ('doc_0', 'contx_4'), ('doc_1', 'contx_5'), ('doc_1', 'contx_6'), ('doc_1', 'contx_7'), ('doc_1', 'contx_8'), ('doc_1', 'contx_9'), ('doc_2', 'contx_10'), ('doc_2', 'contx_11'), ('doc_2', 'contx_12'), ('doc_2', 'contx_13'), ('doc_2', 'contx_14'), ('doc_3', 'contx_15'), ('doc_3', 'contx_16'), ('doc_3', 'contx_17'), ('doc_3', 'contx_18'), ('doc_3', 'contx_19'), ('doc_4', 'contx_20'), ('doc_4', 'contx_21'), ('doc_4', 'contx_22'), ('doc_4', 'contx_23'), ('doc_4', 'contx_24'), ('doc_5', 'contx_25'), ('doc_5', 'contx_26'), ('doc_5', 'contx_27'), ('doc_5', 'contx_28'), ('doc_5', 'contx_29'), ('doc_6', 'contx_30'), ('doc_6', 'contx_31'), ('doc_6', 'contx_32'), ('doc_6', 'contx_33'), ('doc_6', 'contx_34'), ('doc_7', 'contx_35'), ('doc_7', 'contx_36'), ('doc_7', 'contx_37'), ('doc_7', 'contx_38'), ('doc_7', 'contx_39'), ('doc_8', 'contx_40'), ('doc_8', 'contx_41'), ('doc_8', 'contx_42'), ('doc_8', 'contx_43'

#### Test graph Conectivity with networkx

In [ ]:
# Define Bipartide graph
B = nx.Graph()
B.add_nodes_from(document_nodes_test, bipartite=0)
B.add_nodes_from(context_nodes_test, bipartite=1)
B.add_edges_from(edges_test)

time: 3.11 s (started: 2024-10-16 21:12:47 +00:00)


In [ ]:
nx.is_connected(B)

False

time: 508 ms (started: 2024-10-16 21:12:50 +00:00)


In [ ]:
# Number of conected elements
# if == 1 -> all elements of the graph are conected
print(nx.number_connected_components(B))

10
time: 747 ms (started: 2024-10-16 21:12:51 +00:00)


In [ ]:
# size of a cluster
print(len(nx.node_connected_component(B,'doc_0')))

44792
time: 736 ms (started: 2024-10-16 21:12:52 +00:00)


### Creating Graph HeteroData Object

In [ ]:
# Defining nodes, edges and class labels
from torch_geometric.data import HeteroData

data = HeteroData()

# nodes
data['document'].x = document_nodes
data['concept'].x = context_nodes

# edges
data['document', 'has', 'concept'].edge_index = edges

#class labels
data['document'].y = y


time: 912 µs (started: 2024-10-16 21:12:52 +00:00)


In [ ]:
# Setting graph to undirected
data = T.ToUndirected()(data)

time: 9.74 ms (started: 2024-10-16 21:12:52 +00:00)


In [ ]:
# removing duplicate edges
data = T.RemoveDuplicatedEdges()(data)

time: 687 ms (started: 2024-10-16 21:12:52 +00:00)


In [ ]:
# Ensure date in using gpu
data = data.to(device)

time: 34.8 ms (started: 2024-10-16 21:12:53 +00:00)


In [ ]:
data

HeteroData(
  document={
    x=[8817, 384],
    y=[8817],
  },
  concept={ x=[35984, 384] },
  (document, has, concept)={ edge_index=[2, 1104343] },
  (concept, rev_has, document)={ edge_index=[2, 1104343] }
)

time: 7.43 ms (started: 2024-10-16 21:12:53 +00:00)


## TRAINING

### Training rotulated base = 20% ❎

In [ ]:
rotulated_perc = 20

time: 332 µs (started: 2024-10-16 21:12:53 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 61.1458, Train: 0.1055, Test: 0.1002
Early stopping:  nan
Epoch: 002, Loss: 337.4022, Train: 0.2944, Test: 0.2937
Early stopping:  195.34279611632547
Epoch: 003, Loss: 379.3685, Train: 0.1180, Test: 0.1145
Early stopping:  172.88939404578926
Epoch: 004, Loss: 383.5331, Train: 0.0510, Test: 0.0517
Early stopping:  154.2247998004672
Epoch: 005, Loss: 464.8856, Train: 0.1225, Test: 0.1206
Early stopping:  154.69530414284856
Epoch: 006, Loss: 463.4489, Train: 0.0681, Test: 0.0658
Early stopping:  56.31868349834021
Epoch: 007, Loss: 424.4395, Train: 0.0675, Test: 0.0641
Early stopping:  41.39397260041529
Epoch: 008, Loss: 388.9366, Train: 0.3023, Test: 0.2967
Early stopping:  39.017805003863636
Epoch: 009, Loss: 279.5083, Train: 0.2972, Test: 0.2964
Early stopping:  76.45881961173907
Epoch: 010, Loss: 274.3239, Train: 0.1815, Test: 0.1758
Early stopping:  85.61949972190847
Epoch: 011, Loss: 258.5367, Train: 0.1469, Test: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 49.6500, Train: 0.1129, Test: 0.1111
Early stopping:  nan
Epoch: 002, Loss: 261.4294, Train: 0.0391, Test: 0.0424
Early stopping:  149.75063897539252
Epoch: 003, Loss: 390.1833, Train: 0.2796, Test: 0.2841
Early stopping:  171.94528404737233
Epoch: 004, Loss: 250.7992, Train: 0.1299, Test: 0.1297
Early stopping:  140.65117480397635
Epoch: 005, Loss: 320.3696, Train: 0.0868, Test: 0.0815
Early stopping:  127.25370570089842
Epoch: 006, Loss: 273.7035, Train: 0.1202, Test: 0.1072
Early stopping:  57.32671614156111
Epoch: 007, Loss: 218.7594, Train: 0.3125, Test: 0.3116
Early stopping:  66.77127025228512
Epoch: 008, Loss: 166.3525, Train: 0.3137, Test: 0.3098
Early stopping:  57.89456645007269
Epoch: 009, Loss: 150.3812, Train: 0.1134, Test: 0.1120
Early stopping:  71.60596275153706
Epoch: 010, Loss: 143.1105, Train: 0.2666, Test: 0.2580
Early stopping:  55.135717491988125
Epoch: 011, Loss: 102.0418, Train: 0.0766, Test: 0.0763
Early stopping:  42.29371226115113
Epoch: 01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 58.8070, Train: 0.2082, Test: 0.2053
Early stopping:  nan
Epoch: 002, Loss: 154.9793, Train: 0.0743, Test: 0.0773
Early stopping:  68.00411135711987
Epoch: 003, Loss: 508.1554, Train: 0.1061, Test: 0.1072
Early stopping:  236.60674848945087
Epoch: 004, Loss: 617.7375, Train: 0.0136, Test: 0.0159
Early stopping:  269.94649775862297
Epoch: 005, Loss: 643.9929, Train: 0.0635, Test: 0.0652
Early stopping:  271.58529173424284
Epoch: 006, Loss: 519.5800, Train: 0.2995, Test: 0.2976
Early stopping:  195.8749536700454
Epoch: 007, Loss: 420.2508, Train: 0.1367, Test: 0.1337
Early stopping:  90.29077483409463
Epoch: 008, Loss: 443.0203, Train: 0.2978, Test: 0.2991
Early stopping:  100.50538573097965
Epoch: 009, Loss: 408.0964, Train: 0.1622, Test: 0.1622
Early stopping:  97.88217403396555
Epoch: 010, Loss: 413.5359, Train: 0.2575, Test: 0.2461
Early stopping:  45.95186441456756
Epoch: 011, Loss: 352.1096, Train: 0.0868, Test: 0.0795
Early stopping:  33.649345866217594
Epoch: 01

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 49.2478, Train: 0.1140, Test: 0.1150
Early stopping:  nan
Epoch: 002, Loss: 415.3795, Train: 0.2944, Test: 0.2952
Early stopping:  258.8942581439155
Epoch: 003, Loss: 373.3789, Train: 0.0839, Test: 0.0829
Early stopping:  200.36530572506877
Epoch: 004, Loss: 300.5334, Train: 0.0964, Test: 0.0930
Early stopping:  163.94056525328367
Epoch: 005, Loss: 306.6446, Train: 0.0613, Test: 0.0631
Early stopping:  142.31748537100984
Epoch: 006, Loss: 362.9936, Train: 0.1242, Test: 0.1284
Early stopping:  48.21976981576436
Epoch: 007, Loss: 338.4504, Train: 0.2694, Test: 0.2686
Early stopping:  32.59856879766933
Epoch: 008, Loss: 264.8783, Train: 0.2830, Test: 0.2882
Early stopping:  37.563749360503834
Epoch: 009, Loss: 256.5168, Train: 0.2711, Test: 0.2661
Early stopping:  45.93811388737137
Epoch: 010, Loss: 217.9408, Train: 0.2632, Test: 0.2591
Early stopping:  60.42197827251344
Epoch: 011, Loss: 164.4391, Train: 0.1940, Test: 0.1902
Early stopping:  64.0774377653944
Epoch: 012,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 62.1969, Train: 0.2944, Test: 0.2963
Early stopping:  nan
Epoch: 002, Loss: 249.8075, Train: 0.1066, Test: 0.1093
Early stopping:  132.6607055623393
Epoch: 003, Loss: 484.2702, Train: 0.0573, Test: 0.0539
Early stopping:  211.46959496584978
Epoch: 004, Loss: 606.5807, Train: 0.0403, Test: 0.0396
Early stopping:  242.71332707403397
Epoch: 005, Loss: 473.2188, Train: 0.0811, Test: 0.0764
Early stopping:  217.21835620327803
Epoch: 006, Loss: 295.1411, Train: 0.0987, Test: 0.0958
Early stopping:  146.89644276574788
Epoch: 007, Loss: 317.1659, Train: 0.1531, Test: 0.1486
Early stopping:  129.202569977916
Epoch: 008, Loss: 243.3797, Train: 0.1117, Test: 0.0977
Early stopping:  149.6587836995345
Epoch: 009, Loss: 199.1652, Train: 0.3188, Test: 0.3147
Early stopping:  104.32996857021092
Epoch: 010, Loss: 134.2852, Train: 0.3120, Test: 0.3102
Early stopping:  73.86816937635652
Epoch: 011, Loss: 139.8613, Train: 0.3074, Test: 0.3039
Early stopping:  76.34815851469303
Epoch: 012

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 461 ms (started: 2024-10-16 21:15:43 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.5265, Train: 0.2944, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 2.2934, Train: 0.2944, Test: 0.2937
Early stopping:  0.16486346709654756
Epoch: 003, Loss: 2.1768, Train: 0.2944, Test: 0.2937
Early stopping:  0.1780637922302343
Epoch: 004, Loss: 2.1381, Train: 0.3040, Test: 0.3018
Early stopping:  0.174826645004053
Epoch: 005, Loss: 2.0749, Train: 0.3647, Test: 0.3517
Early stopping:  0.17788485092203946
Epoch: 006, Loss: 2.0104, Train: 0.3766, Test: 0.3640
Early stopping:  0.10713488602355686
Epoch: 007, Loss: 1.9462, Train: 0.3755, Test: 0.3597
Early stopping:  0.09346602699669965
Epoch: 008, Loss: 1.8919, Train: 0.3693, Test: 0.3589
Early stopping:  0.09824546245335916
Epoch: 009, Loss: 1.8446, Train: 0.3857, Test: 0.3706
Early stopping:  0.09176138103861087
Epoch: 010, Loss: 1.7883, Train: 0.4226, Test: 0.4016
Early stopping:  0.08640051866912205
Epoch: 011, Loss: 1.7222, Train: 0.4668, Test: 0.4369
Early

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 480 ms (started: 2024-10-16 21:17:54 +00:00)


### Training rotulated base = 40% ❎

In [ ]:
rotulated_perc = 40

time: 470 µs (started: 2024-10-16 21:17:55 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 79.6370, Train: 0.0227, Test: 0.0202
Early stopping:  nan
Epoch: 002, Loss: 322.6305, Train: 0.0635, Test: 0.0639
Early stopping:  171.82235146161983
Epoch: 003, Loss: 383.2128, Train: 0.0712, Test: 0.0652
Early stopping:  160.66234079446485
Epoch: 004, Loss: 405.4357, Train: 0.2944, Test: 0.2947
Early stopping:  149.54644842997007
Epoch: 005, Loss: 338.2818, Train: 0.2893, Test: 0.2854
Early stopping:  130.77465788671057
Epoch: 006, Loss: 272.8007, Train: 0.1069, Test: 0.0977
Early stopping:  52.12861232183127
Epoch: 007, Loss: 278.4528, Train: 0.1262, Test: 0.1225
Early stopping:  59.91815756107001
Epoch: 008, Loss: 220.3753, Train: 0.0763, Test: 0.0737
Early stopping:  70.8506510495985
Epoch: 009, Loss: 230.4311, Train: 0.1466, Test: 0.1427
Early stopping:  46.77432733544935
Epoch: 010, Loss: 154.4404, Train: 0.1339, Test: 0.1213
Early stopping:  49.93295046524715
Epoch: 011, Loss: 131.1138, Train: 0.2896, Test: 0.2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 68.2805, Train: 0.1149, Test: 0.1125
Early stopping:  nan
Epoch: 002, Loss: 542.1357, Train: 0.0763, Test: 0.0765
Early stopping:  335.06625880635346
Epoch: 003, Loss: 651.7943, Train: 0.0499, Test: 0.0474
Early stopping:  310.12150501311834
Epoch: 004, Loss: 607.6354, Train: 0.1095, Test: 0.1040
Early stopping:  269.9067820356782
Epoch: 005, Loss: 403.8134, Train: 0.1086, Test: 0.1092
Early stopping:  235.47286312583893
Epoch: 006, Loss: 278.1821, Train: 0.1171, Test: 0.1164
Early stopping:  153.9507228096973
Epoch: 007, Loss: 127.2587, Train: 0.1146, Test: 0.1119
Early stopping:  220.68716972500516
Epoch: 008, Loss: 56.8997, Train: 0.1438, Test: 0.1283
Early stopping:  220.6200038253517
Epoch: 009, Loss: 24.0395, Train: 0.1049, Test: 0.1015
Early stopping:  159.6652219139419
Epoch: 010, Loss: 6.0467, Train: 0.0893, Test: 0.0843
Early stopping:  110.60660571924369
Epoch: 011, Loss: 4.1722, Train: 0.1200, Test: 0.1096
Early stopping:  51.29092092576878
Epoch: 012, Los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 60.7672, Train: 0.0513, Test: 0.0505
Early stopping:  nan
Epoch: 002, Loss: 288.2207, Train: 0.1358, Test: 0.1327
Early stopping:  160.83386018403752
Epoch: 003, Loss: 334.5786, Train: 0.2093, Test: 0.2049
Early stopping:  146.54734471680848
Epoch: 004, Loss: 440.8289, Train: 0.2941, Test: 0.2939
Early stopping:  160.17750522715096
Epoch: 005, Loss: 451.9881, Train: 0.3029, Test: 0.2958
Early stopping:  158.37692548411215
Epoch: 006, Loss: 298.4916, Train: 0.0686, Test: 0.0637
Early stopping:  78.32163166820658
Epoch: 007, Loss: 308.2779, Train: 0.0760, Test: 0.0739
Early stopping:  73.93641326937744
Epoch: 008, Loss: 199.1404, Train: 0.1151, Test: 0.1164
Early stopping:  106.3933018945527
Epoch: 009, Loss: 181.5539, Train: 0.1202, Test: 0.1196
Early stopping:  107.98159755237779
Epoch: 010, Loss: 127.1104, Train: 0.2405, Test: 0.2281
Early stopping:  78.18749587683266
Epoch: 011, Loss: 49.7233, Train: 0.2544, Test: 0.2493
Early stopping:  95.34077664334059
Epoch: 012

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 75.1803, Train: 0.3154, Test: 0.3062
Early stopping:  nan
Epoch: 002, Loss: 326.7638, Train: 0.0323, Test: 0.0340
Early stopping:  177.8963907277133
Epoch: 003, Loss: 405.3710, Train: 0.0797, Test: 0.0756
Early stopping:  172.48150437270058
Epoch: 004, Loss: 268.2447, Train: 0.1809, Test: 0.1765
Early stopping:  140.83121556432312
Epoch: 005, Loss: 239.6844, Train: 0.2748, Test: 0.2765
Early stopping:  122.66077672012595
Epoch: 006, Loss: 189.9344, Train: 0.0788, Test: 0.0760
Early stopping:  83.06639856034695
Epoch: 007, Loss: 185.8397, Train: 0.0610, Test: 0.0575
Early stopping:  89.44060085876671
Epoch: 008, Loss: 176.4785, Train: 0.1140, Test: 0.1115
Early stopping:  39.88393950987732
Epoch: 009, Loss: 153.8968, Train: 0.1687, Test: 0.1658
Early stopping:  31.49863763328566
Epoch: 010, Loss: 96.2621, Train: 0.1739, Test: 0.1784
Early stopping:  38.51591768116857
Epoch: 011, Loss: 101.0500, Train: 0.2975, Test: 0.3007
Early stopping:  41.88851352637057
Epoch: 012, 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 182.7376, Train: 0.0865, Test: 0.0883
Early stopping:  nan
Epoch: 002, Loss: 200.2208, Train: 0.0961, Test: 0.0960
Early stopping:  12.362521697410351
Epoch: 003, Loss: 297.0530, Train: 0.1024, Test: 0.1047
Early stopping:  61.57672669199641
Epoch: 004, Loss: 504.0166, Train: 0.2941, Test: 0.2945
Early stopping:  147.5059860887377
Epoch: 005, Loss: 565.9239, Train: 0.2706, Test: 0.2727
Early stopping:  175.7541876600924
Epoch: 006, Loss: 442.3615, Train: 0.1937, Test: 0.1898
Early stopping:  150.52918244845483
Epoch: 007, Loss: 351.1262, Train: 0.1047, Test: 0.1092
Early stopping:  109.52234015437544
Epoch: 008, Loss: 272.1595, Train: 0.0851, Test: 0.0900
Early stopping:  117.4730978890026
Epoch: 009, Loss: 263.9707, Train: 0.3088, Test: 0.3083
Early stopping:  126.86480897309768
Epoch: 010, Loss: 206.4316, Train: 0.2130, Test: 0.2070
Early stopping:  91.44534116477585
Epoch: 011, Loss: 194.3599, Train: 0.2292, Test: 0.2259
Early stopping:  62.48446033336068
Epoch: 01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 457 ms (started: 2024-10-16 21:20:43 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.5349, Train: 0.2944, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 2.2907, Train: 0.2944, Test: 0.2943
Early stopping:  0.17268676511713973
Epoch: 003, Loss: 2.1603, Train: 0.2944, Test: 0.2943
Early stopping:  0.19016417184888382
Epoch: 004, Loss: 2.1378, Train: 0.2986, Test: 0.2984
Early stopping:  0.18223976877522258
Epoch: 005, Loss: 2.0677, Train: 0.3599, Test: 0.3525
Early stopping:  0.1844055469448322
Epoch: 006, Loss: 1.9993, Train: 0.4124, Test: 0.3982
Early stopping:  0.10928324167733866
Epoch: 007, Loss: 1.9346, Train: 0.4214, Test: 0.4046
Early stopping:  0.09431104831616102
Epoch: 008, Loss: 1.8744, Train: 0.4132, Test: 0.3975
Early stopping:  0.10438391248713279
Epoch: 009, Loss: 1.8191, Train: 0.4070, Test: 0.3978
Early stopping:  0.09843184336140279
Epoch: 010, Loss: 1.7646, Train: 0.4229, Test: 0.4082
Early stopping:  0.09254425453936702
Epoch: 011, Loss: 1.7057, Train: 0.4478, Test: 0.4304
Ear

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 443 ms (started: 2024-10-16 21:23:17 +00:00)


### Training rotulated base = 60% ❎

In [ ]:
rotulated_perc = 60

time: 425 µs (started: 2024-10-16 21:23:17 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 85.7005, Train: 0.2940, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 431.2759, Train: 0.0588, Test: 0.0536
Early stopping:  244.35874743841907
Epoch: 003, Loss: 291.2050, Train: 0.0690, Test: 0.0692
Early stopping:  173.81713328319566
Epoch: 004, Loss: 365.4099, Train: 0.1681, Test: 0.1636
Early stopping:  149.82112500142875
Epoch: 005, Loss: 390.6188, Train: 0.1236, Test: 0.1211
Early stopping:  136.8399067706771
Epoch: 006, Loss: 367.7355, Train: 0.0491, Test: 0.0499
Early stopping:  51.0193527671637
Epoch: 007, Loss: 334.8643, Train: 0.1191, Test: 0.1154
Early stopping:  38.366107158476595
Epoch: 008, Loss: 205.0591, Train: 0.0853, Test: 0.0831
Early stopping:  74.0758029213539
Epoch: 009, Loss: 138.5168, Train: 0.1909, Test: 0.1945
Early stopping:  109.89425505131518
Epoch: 010, Loss: 80.4778, Train: 0.2921, Test: 0.2952
Early stopping:  123.69991790804804
Epoch: 011, Loss: 76.4372, Train: 0.2524, Test: 0.24

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 77.4738, Train: 0.2898, Test: 0.2889
Early stopping:  nan
Epoch: 002, Loss: 395.6100, Train: 0.1102, Test: 0.1094
Early stopping:  224.95623195761337
Epoch: 003, Loss: 454.6859, Train: 0.0777, Test: 0.0757
Early stopping:  202.89139374723155
Epoch: 004, Loss: 287.9645, Train: 0.1219, Test: 0.1225
Early stopping:  166.0018565394788
Epoch: 005, Loss: 329.0551, Train: 0.0567, Test: 0.0525
Early stopping:  144.20014063565935
Epoch: 006, Loss: 253.0538, Train: 0.0794, Test: 0.0766
Early stopping:  81.47003805216606
Epoch: 007, Loss: 156.0074, Train: 0.0750, Test: 0.0766
Early stopping:  109.29387516998877
Epoch: 008, Loss: 166.0291, Train: 0.0410, Test: 0.0400
Early stopping:  75.68982632275497
Epoch: 009, Loss: 143.5769, Train: 0.1673, Test: 0.1693
Early stopping:  79.51000881469261
Epoch: 010, Loss: 90.4975, Train: 0.2631, Test: 0.2538
Early stopping:  58.72566276223511
Epoch: 011, Loss: 74.3631, Train: 0.2590, Test: 0.2455
Early stopping:  41.04410474755769
Epoch: 012, 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 113.9397, Train: 0.0588, Test: 0.0567
Early stopping:  nan
Epoch: 002, Loss: 387.1398, Train: 0.2904, Test: 0.2898
Early stopping:  193.181613620619
Epoch: 003, Loss: 266.7357, Train: 0.1051, Test: 0.1052
Early stopping:  136.9197008085725
Epoch: 004, Loss: 311.3484, Train: 0.0813, Test: 0.0780
Early stopping:  115.17624751835325
Epoch: 005, Loss: 197.4960, Train: 0.0675, Test: 0.0663
Early stopping:  104.85459420856719
Epoch: 006, Loss: 271.9218, Train: 0.0667, Test: 0.0663
Early stopping:  69.4077596119756
Epoch: 007, Loss: 250.9160, Train: 0.1236, Test: 0.1228
Early stopping:  41.270203461533754
Epoch: 008, Loss: 246.2799, Train: 0.1758, Test: 0.1815
Early stopping:  41.41001185305531
Epoch: 009, Loss: 245.2961, Train: 0.3023, Test: 0.3071
Early stopping:  27.312380730808307
Epoch: 010, Loss: 264.1449, Train: 0.3011, Test: 0.3017
Early stopping:  11.772116450428918
Epoch: 011, Loss: 264.4868, Train: 0.2560, Test: 0.2583
Early stopping:  9.453980973909891
Epoch: 012

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 141.9694, Train: 0.2940, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 195.8751, Train: 0.1104, Test: 0.1120
Early stopping:  38.11709728757467
Epoch: 003, Loss: 462.2955, Train: 0.1221, Test: 0.1256
Early stopping:  171.5101719540258
Epoch: 004, Loss: 373.9397, Train: 0.0904, Test: 0.0970
Early stopping:  149.94954671252108
Epoch: 005, Loss: 315.6813, Train: 0.0749, Test: 0.0825
Early stopping:  130.23776570999408
Epoch: 006, Loss: 291.5570, Train: 0.0516, Test: 0.0536
Early stopping:  98.84643791596356
Epoch: 007, Loss: 299.0990, Train: 0.0856, Test: 0.0842
Early stopping:  71.33830408688782
Epoch: 008, Loss: 197.4769, Train: 0.0745, Test: 0.0763
Early stopping:  63.63387855883763
Epoch: 009, Loss: 105.1909, Train: 0.3076, Test: 0.3085
Early stopping:  89.22439843616016
Epoch: 010, Loss: 89.3109, Train: 0.2858, Test: 0.2949
Early stopping:  99.23482120933127
Epoch: 011, Loss: 82.7275, Train: 0.2268, Test: 0.2248
Early stopping:  92.98904734257182
Epoch: 012, L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 63.0405, Train: 0.2943, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 453.5551, Train: 0.0335, Test: 0.0357
Early stopping:  276.13554797064194
Epoch: 003, Loss: 664.6469, Train: 0.1163, Test: 0.1148
Early stopping:  305.2298670618461
Epoch: 004, Loss: 489.1987, Train: 0.1138, Test: 0.1128
Early stopping:  253.74773596308887
Epoch: 005, Loss: 406.7010, Train: 0.1113, Test: 0.1117
Early stopping:  219.8061368156803
Epoch: 006, Loss: 459.8578, Train: 0.1227, Test: 0.1216
Early stopping:  99.45207747159454
Epoch: 007, Loss: 315.9105, Train: 0.0739, Test: 0.0737
Early stopping:  128.5221157241936
Epoch: 008, Loss: 289.1974, Train: 0.2890, Test: 0.2864
Early stopping:  87.50173597545209
Epoch: 009, Loss: 231.7673, Train: 0.2941, Test: 0.2932
Early stopping:  91.77155540604575
Epoch: 010, Loss: 237.8791, Train: 0.2905, Test: 0.2841
Early stopping:  92.46549300162432
Epoch: 011, Loss: 202.7789, Train: 0.2794, Test: 0.2753
Early stopping:  45.9298313197084
Epoch: 012, L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 462 ms (started: 2024-10-16 21:25:40 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4947, Train: 0.2943, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 2.2598, Train: 0.2943, Test: 0.2943
Early stopping:  0.16610831641416535
Epoch: 003, Loss: 2.1692, Train: 0.2943, Test: 0.2943
Early stopping:  0.16798152112172923
Epoch: 004, Loss: 2.1185, Train: 0.3079, Test: 0.3099
Early stopping:  0.16667945080533791
Epoch: 005, Loss: 2.0690, Train: 0.3484, Test: 0.3419
Early stopping:  0.16785197664113266
Epoch: 006, Loss: 2.0245, Train: 0.3558, Test: 0.3519
Early stopping:  0.09129730094661055
Epoch: 007, Loss: 1.9698, Train: 0.3493, Test: 0.3470
Early stopping:  0.07795430496476206
Epoch: 008, Loss: 1.9120, Train: 0.3495, Test: 0.3419
Early stopping:  0.08108033664837107
Epoch: 009, Loss: 1.8587, Train: 0.3578, Test: 0.3502
Early stopping:  0.08436433456421047
Epoch: 010, Loss: 1.8075, Train: 0.3811, Test: 0.3748
Early stopping:  0.08622075347826913
Epoch: 011, Loss: 1.7516, Train: 0.4208, Test: 0.4088
Ea

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 439 ms (started: 2024-10-16 21:28:16 +00:00)


### Training rotulated base = 80% ❎

In [ ]:
rotulated_perc = 80

time: 469 µs (started: 2024-10-16 21:28:17 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 78.0120, Train: 0.2943, Test: 0.2942
Early stopping:  nan
Epoch: 002, Loss: 467.8694, Train: 0.0125, Test: 0.0113
Early stopping:  275.67082130892055
Epoch: 003, Loss: 615.2813, Train: 0.0651, Test: 0.0680
Early stopping:  277.602060422929
Epoch: 004, Loss: 677.0381, Train: 0.1136, Test: 0.1139
Early stopping:  269.06863366291844
Epoch: 005, Loss: 751.7266, Train: 0.1114, Test: 0.1083
Early stopping:  267.1551255446567
Epoch: 006, Loss: 602.4797, Train: 0.0471, Test: 0.0482
Early stopping:  104.92468110680626
Epoch: 007, Loss: 443.4893, Train: 0.0934, Test: 0.0952
Early stopping:  114.09489063503925
Epoch: 008, Loss: 231.3313, Train: 0.2985, Test: 0.3010
Early stopping:  207.4214743246863
Epoch: 009, Loss: 221.0394, Train: 0.2951, Test: 0.2993
Early stopping:  231.60871888233552
Epoch: 010, Loss: 281.2063, Train: 0.2934, Test: 0.2902
Early stopping:  164.1320580424752
Epoch: 011, Loss: 271.8357, Train: 0.2406, Test: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 41.7350, Train: 0.2929, Test: 0.2920
Early stopping:  nan
Epoch: 002, Loss: 524.5143, Train: 0.1224, Test: 0.1241
Early stopping:  341.3765200315825
Epoch: 003, Loss: 756.6544, Train: 0.1110, Test: 0.1088
Early stopping:  364.70866824463985
Epoch: 004, Loss: 839.3804, Train: 0.0825, Test: 0.0794
Early stopping:  358.2709527482591
Epoch: 005, Loss: 702.5474, Train: 0.0540, Test: 0.0488
Early stopping:  318.61548347073375
Epoch: 006, Loss: 549.3184, Train: 0.0749, Test: 0.0709
Early stopping:  134.9885894099055
Epoch: 007, Loss: 444.2590, Train: 0.0289, Test: 0.0244
Early stopping:  159.77481834085998
Epoch: 008, Loss: 371.8979, Train: 0.0630, Test: 0.0527
Early stopping:  190.36712153936398
Epoch: 009, Loss: 263.9988, Train: 0.0471, Test: 0.0363
Early stopping:  168.0983087187419
Epoch: 010, Loss: 151.1736, Train: 0.0533, Test: 0.0448
Early stopping:  154.78405529183232
Epoch: 011, Loss: 80.5393, Train: 0.0553, Test: 0.0505
Early stopping:  150.42414595414218
Epoch: 01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 66.5985, Train: 0.2941, Test: 0.2931
Early stopping:  nan
Epoch: 002, Loss: 409.7770, Train: 0.0547, Test: 0.0561
Early stopping:  242.6638372136626
Epoch: 003, Loss: 262.1969, Train: 0.1103, Test: 0.1071
Early stopping:  172.1482374034643
Epoch: 004, Loss: 300.9800, Train: 0.0767, Test: 0.0771
Early stopping:  143.2031429597035
Epoch: 005, Loss: 320.2294, Train: 0.1293, Test: 0.1219
Early stopping:  126.91953910010615
Epoch: 006, Loss: 204.3776, Train: 0.0936, Test: 0.0884
Early stopping:  75.8826517024421
Epoch: 007, Loss: 137.4511, Train: 0.1338, Test: 0.1310
Early stopping:  74.6757952179252
Epoch: 008, Loss: 89.2679, Train: 0.3131, Test: 0.3044
Early stopping:  100.37181915242672
Epoch: 009, Loss: 81.0931, Train: 0.2989, Test: 0.2920
Early stopping:  98.95757400469385
Epoch: 010, Loss: 64.8056, Train: 0.2573, Test: 0.2523
Early stopping:  56.619131017472704
Epoch: 011, Loss: 43.7190, Train: 0.2379, Test: 0.2194
Early stopping:  34.933933022601714
Epoch: 012, Loss

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 70.9015, Train: 0.2919, Test: 0.2868
Early stopping:  nan
Epoch: 002, Loss: 218.3100, Train: 0.1139, Test: 0.1066
Early stopping:  104.23357200846573
Epoch: 003, Loss: 453.6166, Train: 0.1214, Test: 0.1202
Early stopping:  193.0325259054032
Epoch: 004, Loss: 572.5329, Train: 0.1119, Test: 0.1128
Early stopping:  226.3511929558384
Epoch: 005, Loss: 744.7433, Train: 0.0423, Test: 0.0420
Early stopping:  270.22439593538127
Epoch: 006, Loss: 800.4385, Train: 0.0627, Test: 0.0618
Early stopping:  234.59853026805868
Epoch: 007, Loss: 639.7764, Train: 0.1029, Test: 0.0998
Early stopping:  137.82021366348312
Epoch: 008, Loss: 359.1432, Train: 0.0981, Test: 0.0947
Early stopping:  172.3035871824876
Epoch: 009, Loss: 256.8231, Train: 0.2948, Test: 0.2863
Early stopping:  240.08569141532013
Epoch: 010, Loss: 239.9736, Train: 0.2951, Test: 0.2851
Early stopping:  249.03793026004558
Epoch: 011, Loss: 223.4921, Train: 0.2850, Test: 0.2772
Early stopping:  173.67810882568833
Epoch: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 60.6949, Train: 0.2942, Test: 0.2931
Early stopping:  nan
Epoch: 002, Loss: 365.9745, Train: 0.1099, Test: 0.1060
Early stopping:  215.86525518885165
Epoch: 003, Loss: 430.6253, Train: 0.0535, Test: 0.0482
Early stopping:  197.57861919433736
Epoch: 004, Loss: 418.5889, Train: 0.0637, Test: 0.0646
Early stopping:  174.45751502423178
Epoch: 005, Loss: 373.7447, Train: 0.0442, Test: 0.0431
Early stopping:  153.05750985488274
Epoch: 006, Loss: 296.8939, Train: 0.1157, Test: 0.1173
Early stopping:  52.80701091649633
Epoch: 007, Loss: 159.4060, Train: 0.2812, Test: 0.2704
Early stopping:  111.70736967298083
Epoch: 008, Loss: 172.2354, Train: 0.2709, Test: 0.2585
Early stopping:  116.56474736583567
Epoch: 009, Loss: 169.4147, Train: 0.1042, Test: 0.0941
Early stopping:  96.22113473623062
Epoch: 010, Loss: 164.8918, Train: 0.1163, Test: 0.1105
Early stopping:  58.52151104081021
Epoch: 011, Loss: 135.5757, Train: 0.1385, Test: 0.1372
Early stopping:  14.65182474460769
Epoch: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 495 ms (started: 2024-10-16 21:30:56 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.5447, Train: 0.2941, Test: 0.2931
Early stopping:  nan
Epoch: 002, Loss: 2.3165, Train: 0.2943, Test: 0.2942
Early stopping:  0.1613932641775546
Epoch: 003, Loss: 2.1832, Train: 0.2943, Test: 0.2942
Early stopping:  0.18284030676730956
Epoch: 004, Loss: 2.1538, Train: 0.2969, Test: 0.2948
Early stopping:  0.17811434631511947
Epoch: 005, Loss: 2.0941, Train: 0.3311, Test: 0.3243
Early stopping:  0.1795338268576027
Epoch: 006, Loss: 2.0250, Train: 0.3864, Test: 0.3793
Early stopping:  0.10894765417976568
Epoch: 007, Loss: 1.9622, Train: 0.3995, Test: 0.3866
Early stopping:  0.09101145175841528
Epoch: 008, Loss: 1.9038, Train: 0.4010, Test: 0.3815
Early stopping:  0.09997347188353503
Epoch: 009, Loss: 1.8468, Train: 0.4037, Test: 0.3832
Early stopping:  0.09744962201270842
Epoch: 010, Loss: 1.7907, Train: 0.4201, Test: 0.4025
Early stopping:  0.09234401652983311
Epoch: 011, Loss: 1.7320, Train: 0.4517, Test: 0.4456
Earl

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 466 ms (started: 2024-10-16 21:33:19 +00:00)


--------------------------------------
# Hetrogenous Graph for Keyphrase = 2 or 3
--------------------------------------

In [ ]:
print(df["class"].unique())
class_number = len(df["class"].unique())
print(class_number)

['basic_materials' 'capital_goods' 'conglomerates_industry'
 'consumer_cyclical' 'consumer_non-cyclical' 'energy' 'financial'
 'healthcare' 'services' 'technology' 'transportation' 'utilities']
12
time: 2.91 ms (started: 2024-10-16 21:33:20 +00:00)


In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase23"

model_name = dataset_name+"_"+keyphrase

time: 467 µs (started: 2024-10-16 21:33:20 +00:00)


## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

document_nodes = []
context_nodes = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

time: 18.1 s (started: 2024-10-16 21:33:20 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 8817
number of context nodes: 35753
number of shared contexts: 7655
number of direct edges (first dimension): 43408
number of direct edges (second dimension): 43408
time: 901 µs (started: 2024-10-16 21:33:38 +00:00)


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

time: 2min 2s (started: 2024-10-16 21:33:38 +00:00)


##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

time: 217 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

time: 50.5 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# Show documents nodes
document_nodes

tensor([[-5.8602e-02,  1.4557e-02,  1.7552e-02,  ..., -1.8911e-02,
         -1.8681e-02, -2.2412e-02],
        [-1.2433e-01, -1.2993e-02,  1.4001e-02,  ..., -1.7944e-02,
          1.1914e-02, -8.1062e-03],
        [-8.5865e-02,  4.3530e-02, -7.1712e-02,  ...,  5.6326e-02,
         -4.5610e-02,  3.1375e-02],
        ...,
        [-1.6584e-02, -3.9770e-02, -2.2835e-02,  ..., -4.2101e-05,
          1.2257e-02,  2.9048e-02],
        [ 3.1864e-02, -1.0648e-02, -3.8576e-02,  ..., -4.5610e-03,
         -1.7202e-02,  4.2433e-02],
        [ 4.3771e-02,  3.8128e-02,  2.0526e-02,  ..., -1.0110e-02,
          3.8847e-02,  4.4935e-02]])

time: 5.52 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# Quantity of document nodes
len(document_nodes)

8817

time: 3.11 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# Show context nodes
context_nodes

tensor([[-0.0714,  0.0858, -0.0434,  ...,  0.0149,  0.0713,  0.0182],
        [-0.0430,  0.0382,  0.0263,  ...,  0.0298,  0.0570, -0.0218],
        [-0.0816,  0.0714, -0.0450,  ...,  0.0140,  0.0811,  0.0313],
        ...,
        [-0.0202,  0.0236, -0.0472,  ..., -0.0497,  0.0594,  0.0337],
        [-0.0464,  0.0437,  0.0029,  ...,  0.0401,  0.0498,  0.0721],
        [-0.0457, -0.0080, -0.0750,  ...,  0.0681, -0.0170,  0.0555]])

time: 4.71 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# Quantity of context nodes
len(context_nodes)

35753

time: 4.07 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

1947013
1947013
time: 1.01 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  8816,  8816,  8816],
        [    0,     1,     2,  ..., 19486, 31355, 23119]])
time: 1.4 ms (started: 2024-10-16 21:35:40 +00:00)


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['basic_materials' 'capital_goods' 'conglomerates_industry'
 'consumer_cyclical' 'consumer_non-cyclical' 'energy' 'financial'
 'healthcare' 'services' 'technology' 'transportation' 'utilities']
time: 3.15 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'basic_materials': 0, 'capital_goods': 1, 'conglomerates_industry': 2, 'consumer_cyclical': 3, 'consumer_non-cyclical': 4, 'energy': 5, 'financial': 6, 'healthcare': 7, 'services': 8, 'technology': 9, 'transportation': 10, 'utilities': 11} 



<ipython-input-331-a018e46c97dd>:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  labels["class"][i] = dit[labels["class"][i]]
<ipython-input-331-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,class
0,0
1,0
2,0
3,0
4,0


time: 883 ms (started: 2024-10-16 21:35:40 +00:00)


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([ 0,  0,  0,  ..., 11, 11, 11])

time: 9.2 ms (started: 2024-10-16 21:35:41 +00:00)


In [ ]:
len(y)

8817

time: 2.3 ms (started: 2024-10-16 21:35:41 +00:00)


### Testing Graph with Networkx




#### Defining overal graph in networkx maner

In [ ]:
# Run graph Representation for networkx
all_contexts_list =[]
edges_test = []
cont = 0
new_edge_cont = 0

document_nodes_test = []
context_nodes_test = []

# Creating graph nodes and edges
for i in range(len(df)):
    # adding new documents for every node
    document_nodes_test.append("doc_"+str(i)) # in the actual graph nodes -> documents embeddings

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list:
            # add a new context node for every new keyphrase in df
            context_nodes_test.append("contx_"+str(new_edge_cont)) # in the actual graph nodes -> context embeddings

            # add a new edge between doc and new context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(new_edge_cont)))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges_test.append(("doc_"+str(df["id"][i]),"contx_"+str(all_contexts_list.index(key[0]))))
            cont+=1

time: 14.4 s (started: 2024-10-16 21:35:41 +00:00)


In [ ]:
edges_test = [("doc_"+str(i[0]),"contx_"+str(i[1])) for i in edges_tuple]

time: 1.28 s (started: 2024-10-16 21:35:56 +00:00)


In [ ]:
print("number of document nodes:",len(document_nodes_test))
print("number of context nodes:",len(context_nodes_test))
print("number of edges:",len(edges_test))

number of document nodes: 8817
number of context nodes: 35753
number of edges: 1947013
time: 854 µs (started: 2024-10-16 21:35:57 +00:00)


In [ ]:
print(edges_test[0:100])

[('doc_0', 'contx_0'), ('doc_0', 'contx_1'), ('doc_0', 'contx_2'), ('doc_0', 'contx_3'), ('doc_0', 'contx_4'), ('doc_1', 'contx_5'), ('doc_1', 'contx_6'), ('doc_1', 'contx_7'), ('doc_1', 'contx_8'), ('doc_1', 'contx_9'), ('doc_2', 'contx_10'), ('doc_2', 'contx_11'), ('doc_2', 'contx_12'), ('doc_2', 'contx_13'), ('doc_2', 'contx_14'), ('doc_3', 'contx_15'), ('doc_3', 'contx_16'), ('doc_3', 'contx_17'), ('doc_3', 'contx_18'), ('doc_3', 'contx_19'), ('doc_4', 'contx_20'), ('doc_4', 'contx_21'), ('doc_4', 'contx_22'), ('doc_4', 'contx_23'), ('doc_4', 'contx_24'), ('doc_5', 'contx_25'), ('doc_5', 'contx_26'), ('doc_5', 'contx_27'), ('doc_5', 'contx_28'), ('doc_5', 'contx_29'), ('doc_6', 'contx_30'), ('doc_6', 'contx_31'), ('doc_6', 'contx_32'), ('doc_6', 'contx_33'), ('doc_6', 'contx_34'), ('doc_7', 'contx_35'), ('doc_7', 'contx_36'), ('doc_7', 'contx_37'), ('doc_7', 'contx_38'), ('doc_7', 'contx_39'), ('doc_8', 'contx_40'), ('doc_8', 'contx_41'), ('doc_8', 'contx_42'), ('doc_8', 'contx_43'

#### Test graph Conectivity with networkx

In [ ]:
# Define Bipartide graph
B = nx.Graph()
B.add_nodes_from(document_nodes_test, bipartite=0)
B.add_nodes_from(context_nodes_test, bipartite=1)
B.add_edges_from(edges_test)

time: 3.49 s (started: 2024-10-16 21:35:57 +00:00)


In [ ]:
nx.is_connected(B)

False

time: 555 ms (started: 2024-10-16 21:36:01 +00:00)


In [ ]:
# Number of conected elements
# if == 1 -> all elements of the graph are conected
print(nx.number_connected_components(B))

10
time: 568 ms (started: 2024-10-16 21:36:01 +00:00)


In [ ]:
# size of a cluster
print(len(nx.node_connected_component(B,'doc_0')))

44561
time: 560 ms (started: 2024-10-16 21:36:02 +00:00)


### Creating Graph HeteroData Object

In [ ]:
# Defining nodes, edges and class labels
from torch_geometric.data import HeteroData

data = HeteroData()

# nodes
data['document'].x = document_nodes
data['concept'].x = context_nodes

# edges
data['document', 'has', 'concept'].edge_index = edges

#class labels
data['document'].y = y


time: 801 µs (started: 2024-10-16 21:36:02 +00:00)


In [ ]:
# Setting graph to undirected
data = T.ToUndirected()(data)

time: 11.6 ms (started: 2024-10-16 21:36:02 +00:00)


In [ ]:
# removing duplicate edges
data = T.RemoveDuplicatedEdges()(data)

time: 528 ms (started: 2024-10-16 21:36:02 +00:00)


In [ ]:
# Ensure date in using gpu
data = data.to(device)

time: 25.2 ms (started: 2024-10-16 21:36:03 +00:00)


In [ ]:
data

HeteroData(
  document={
    x=[8817, 384],
    y=[8817],
  },
  concept={ x=[35753, 384] },
  (document, has, concept)={ edge_index=[2, 1094600] },
  (concept, rev_has, document)={ edge_index=[2, 1094600] }
)

time: 3.24 ms (started: 2024-10-16 21:36:03 +00:00)


## TRAINING

### Training rotulated base = 20% ❎

In [ ]:
rotulated_perc = 20

time: 315 µs (started: 2024-10-16 21:36:03 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 41.5275, Train: 0.1151, Test: 0.1110
Early stopping:  nan
Epoch: 002, Loss: 343.6941, Train: 0.2938, Test: 0.2872
Early stopping:  213.66409725026958
Epoch: 003, Loss: 260.1060, Train: 0.0766, Test: 0.0747
Early stopping:  156.02791371831276
Epoch: 004, Loss: 415.5328, Train: 0.1095, Test: 0.1109
Early stopping:  162.0870479137794
Epoch: 005, Loss: 382.2715, Train: 0.0584, Test: 0.0608
Early stopping:  149.81521062715237
Epoch: 006, Loss: 340.3053, Train: 0.0510, Test: 0.0495
Early stopping:  58.175556200698445
Epoch: 007, Loss: 253.0544, Train: 0.0664, Test: 0.0671
Early stopping:  72.38756780212263
Epoch: 008, Loss: 195.6918, Train: 0.2138, Test: 0.2193
Early stopping:  91.26369758152003
Epoch: 009, Loss: 146.9955, Train: 0.0976, Test: 0.0920
Early stopping:  97.83397852125599
Epoch: 010, Loss: 153.1552, Train: 0.1934, Test: 0.1868
Early stopping:  80.51616506109869
Epoch: 011, Loss: 119.6956, Train: 0.3199, Test: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 55.7783, Train: 0.2944, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 348.6227, Train: 0.1112, Test: 0.1145
Early stopping:  207.07222224723355
Epoch: 003, Loss: 456.6776, Train: 0.1197, Test: 0.1182
Early stopping:  207.42623052619845
Epoch: 004, Loss: 592.4072, Train: 0.0675, Test: 0.0672
Early stopping:  228.03101964580753
Epoch: 005, Loss: 556.1412, Train: 0.0584, Test: 0.0576
Early stopping:  215.47771126979845
Epoch: 006, Loss: 505.7266, Train: 0.1106, Test: 0.1092
Early stopping:  95.1167760151573
Epoch: 007, Loss: 425.8822, Train: 0.0340, Test: 0.0339
Early stopping:  68.59780091519772
Epoch: 008, Loss: 313.3448, Train: 0.3012, Test: 0.3010
Early stopping:  111.56718748888864
Epoch: 009, Loss: 234.0000, Train: 0.2672, Test: 0.2705
Early stopping:  133.27310378058095
Epoch: 010, Loss: 196.0497, Train: 0.2711, Test: 0.2681
Early stopping:  129.792415549986
Epoch: 011, Loss: 132.5934, Train: 0.0709, Test: 0.0662
Early stopping:  113.31366960304553
Epoch: 01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 60.1007, Train: 0.2989, Test: 0.2875
Early stopping:  nan
Epoch: 002, Loss: 197.2133, Train: 0.0874, Test: 0.0849
Early stopping:  96.95331826067267
Epoch: 003, Loss: 291.4950, Train: 0.1072, Test: 0.1067
Early stopping:  116.35596485296387
Epoch: 004, Loss: 381.2415, Train: 0.0726, Test: 0.0688
Early stopping:  137.32092904607725
Epoch: 005, Loss: 392.6317, Train: 0.0425, Test: 0.0418
Early stopping:  138.8178648121147
Epoch: 006, Loss: 374.2296, Train: 0.1123, Test: 0.1087
Early stopping:  83.04337303673562
Epoch: 007, Loss: 315.9588, Train: 0.1140, Test: 0.1138
Early stopping:  44.59826458720362
Epoch: 008, Loss: 205.6164, Train: 0.2450, Test: 0.2441
Early stopping:  77.6138627006189
Epoch: 009, Loss: 100.6198, Train: 0.2689, Test: 0.2655
Early stopping:  123.02882020944807
Epoch: 010, Loss: 93.2674, Train: 0.2246, Test: 0.2126
Early stopping:  125.98935906674977
Epoch: 011, Loss: 78.1750, Train: 0.2507, Test: 0.2474
Early stopping:  101.32928018786856
Epoch: 012, 

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 89.3850, Train: 0.2944, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 348.3806, Train: 0.1163, Test: 0.1181
Early stopping:  183.13752195048576
Epoch: 003, Loss: 494.8129, Train: 0.1378, Test: 0.1320
Early stopping:  205.30176620212927
Epoch: 004, Loss: 489.0871, Train: 0.1117, Test: 0.1106
Early stopping:  189.84330188251812
Epoch: 005, Loss: 471.8105, Train: 0.0357, Test: 0.0353
Early stopping:  172.45252889295713
Epoch: 006, Loss: 444.1104, Train: 0.0601, Test: 0.0593
Early stopping:  59.94213742847242
Epoch: 007, Loss: 394.0081, Train: 0.0754, Test: 0.0739
Early stopping:  41.22283567204216
Epoch: 008, Loss: 266.2455, Train: 0.0550, Test: 0.0537
Early stopping:  89.60098840507975
Epoch: 009, Loss: 220.9707, Train: 0.2734, Test: 0.2713
Early stopping:  110.51030438726426
Epoch: 010, Loss: 110.3948, Train: 0.1106, Test: 0.1060
Early stopping:  134.27053463927953
Epoch: 011, Loss: 114.0173, Train: 0.1492, Test: 0.1456
Early stopping:  117.95938158979537
Epoch: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 116.5276, Train: 0.1089, Test: 0.1089
Early stopping:  nan
Epoch: 002, Loss: 445.1912, Train: 0.2898, Test: 0.2898
Early stopping:  232.40029061225576
Epoch: 003, Loss: 331.5057, Train: 0.0250, Test: 0.0221
Early stopping:  166.9130420644995
Epoch: 004, Loss: 404.1273, Train: 0.1344, Test: 0.1317
Early stopping:  146.2969284904127
Epoch: 005, Loss: 333.1706, Train: 0.0874, Test: 0.0843
Early stopping:  126.75841872374211
Epoch: 006, Loss: 351.0881, Train: 0.1078, Test: 0.0995
Early stopping:  49.921849645766194
Epoch: 007, Loss: 328.1021, Train: 0.1412, Test: 0.1308
Early stopping:  31.760518252055363
Epoch: 008, Loss: 273.5945, Train: 0.3165, Test: 0.3130
Early stopping:  46.94005881398564
Epoch: 009, Loss: 208.5713, Train: 0.3069, Test: 0.2944
Early stopping:  58.2032707022713
Epoch: 010, Loss: 192.4031, Train: 0.2813, Test: 0.2794
Early stopping:  70.27802071816924
Epoch: 011, Loss: 146.6178, Train: 0.1248, Test: 0.1225
Early stopping:  71.32622202595215
Epoch: 012

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 432 ms (started: 2024-10-16 21:39:14 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4893, Train: 0.2944, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 2.2951, Train: 0.2944, Test: 0.2943
Early stopping:  0.13734022631978143
Epoch: 003, Loss: 2.1681, Train: 0.2944, Test: 0.2943
Early stopping:  0.16177291560517465
Epoch: 004, Loss: 2.1185, Train: 0.3103, Test: 0.3073
Early stopping:  0.16537562262715663
Epoch: 005, Loss: 2.0560, Train: 0.3681, Test: 0.3578
Early stopping:  0.1716979902216828
Epoch: 006, Loss: 2.0008, Train: 0.4084, Test: 0.3901
Early stopping:  0.11288105895967508
Epoch: 007, Loss: 1.9310, Train: 0.4090, Test: 0.3914
Early stopping:  0.09370945089202214
Epoch: 008, Loss: 1.8553, Train: 0.3993, Test: 0.3833
Early stopping:  0.1031492202542417
Epoch: 009, Loss: 1.7902, Train: 0.4095, Test: 0.3910
Early stopping:  0.10716939426699298
Epoch: 010, Loss: 1.7305, Train: 0.4441, Test: 0.4249
Early stopping:  0.10784884261508029
Epoch: 011, Loss: 1.6631, Train: 0.4963, Test: 0.4651
Earl

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 467 ms (started: 2024-10-16 21:41:23 +00:00)


### Training rotulated base = 40% ❎

In [ ]:
rotulated_perc = 40

time: 379 µs (started: 2024-10-16 21:41:24 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 55.5688, Train: 0.1129, Test: 0.1119
Early stopping:  nan
Epoch: 002, Loss: 400.0464, Train: 0.2927, Test: 0.2924
Early stopping:  243.58238765315772
Epoch: 003, Loss: 465.3965, Train: 0.1438, Test: 0.1344
Early stopping:  220.18708919025462
Epoch: 004, Loss: 323.3477, Train: 0.0967, Test: 0.0830
Early stopping:  179.96763336550976
Epoch: 005, Loss: 363.8507, Train: 0.0536, Test: 0.0478
Early stopping:  157.63249536211282
Epoch: 006, Loss: 387.6730, Train: 0.1710, Test: 0.1690
Early stopping:  52.198027434287056
Epoch: 007, Loss: 338.8671, Train: 0.2782, Test: 0.2633
Early stopping:  55.73726914574455
Epoch: 008, Loss: 327.1887, Train: 0.1393, Test: 0.1329
Early stopping:  27.153626626867418
Epoch: 009, Loss: 302.4108, Train: 0.2998, Test: 0.2939
Early stopping:  32.93937187967281
Epoch: 010, Loss: 250.6355, Train: 0.2918, Test: 0.2826
Early stopping:  50.24668140834111
Epoch: 011, Loss: 165.8226, Train: 0.1066, Test: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 74.2253, Train: 0.2941, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 457.5911, Train: 0.0474, Test: 0.0478
Early stopping:  271.080496769919
Epoch: 003, Loss: 530.7856, Train: 0.0584, Test: 0.0590
Early stopping:  245.2121382292879
Epoch: 004, Loss: 425.1281, Train: 0.1072, Test: 0.1094
Early stopping:  203.33143368265755
Epoch: 005, Loss: 367.3201, Train: 0.0771, Test: 0.0713
Early stopping:  176.10226817476826
Epoch: 006, Loss: 281.8610, Train: 0.1208, Test: 0.1176
Early stopping:  93.93070881988847
Epoch: 007, Loss: 271.3114, Train: 0.1228, Test: 0.1092
Early stopping:  107.54854564675763
Epoch: 008, Loss: 170.4464, Train: 0.3003, Test: 0.2916
Early stopping:  97.56229847220987
Epoch: 009, Loss: 113.4060, Train: 0.3120, Test: 0.3096
Early stopping:  99.75360143453369
Epoch: 010, Loss: 127.2753, Train: 0.1761, Test: 0.1705
Early stopping:  79.36067323016624
Epoch: 011, Loss: 124.3598, Train: 0.2847, Test: 0.2873
Early stopping:  65.1963234608159
Epoch: 012, L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 52.9752, Train: 0.1390, Test: 0.1389
Early stopping:  nan
Epoch: 002, Loss: 607.4414, Train: 0.0343, Test: 0.0370
Early stopping:  392.06684358774066
Epoch: 003, Loss: 410.4365, Train: 0.1140, Test: 0.1123
Early stopping:  281.07601321613134
Epoch: 004, Loss: 264.9410, Train: 0.0766, Test: 0.0758
Early stopping:  234.0632639848149
Epoch: 005, Loss: 180.6722, Train: 0.0794, Test: 0.0739
Early stopping:  213.98114650451728
Epoch: 006, Loss: 125.0816, Train: 0.2124, Test: 0.2119
Early stopping:  194.43000615076255
Epoch: 007, Loss: 80.7760, Train: 0.1441, Test: 0.1486
Early stopping:  130.3217473073717
Epoch: 008, Loss: 77.2056, Train: 0.2323, Test: 0.2461
Early stopping:  78.69751208442399
Epoch: 009, Loss: 59.9313, Train: 0.2535, Test: 0.2663
Early stopping:  48.78454151869494
Epoch: 010, Loss: 40.9503, Train: 0.1946, Test: 0.2019
Early stopping:  31.29231624355598
Epoch: 011, Loss: 27.2683, Train: 0.3035, Test: 0.3115
Early stopping:  23.04053413041319
Epoch: 012, Los

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 77.8321, Train: 0.2941, Test: 0.2937
Early stopping:  nan
Epoch: 002, Loss: 447.1064, Train: 0.1234, Test: 0.1225
Early stopping:  261.11639914390986
Epoch: 003, Loss: 322.9109, Train: 0.1566, Test: 0.1493
Early stopping:  187.905878605781
Epoch: 004, Loss: 374.3645, Train: 0.0675, Test: 0.0643
Early stopping:  160.13588678422363
Epoch: 005, Loss: 447.1007, Train: 0.0627, Test: 0.0618
Early stopping:  152.44588532406507
Epoch: 006, Loss: 402.8055, Train: 0.0604, Test: 0.0592
Early stopping:  52.53269972219188
Epoch: 007, Loss: 318.8773, Train: 0.0519, Test: 0.0505
Early stopping:  54.357812289155234
Epoch: 008, Loss: 248.2870, Train: 0.2981, Test: 0.3003
Early stopping:  77.1060255695611
Epoch: 009, Loss: 222.8594, Train: 0.2921, Test: 0.2894
Early stopping:  96.53075653432167
Epoch: 010, Loss: 188.9616, Train: 0.2867, Test: 0.2880
Early stopping:  85.27170879847907
Epoch: 011, Loss: 147.4758, Train: 0.1693, Test: 0.1618
Early stopping:  64.5526371572904
Epoch: 012, L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 65.8327, Train: 0.2629, Test: 0.2586
Early stopping:  nan
Epoch: 002, Loss: 284.4404, Train: 0.0834, Test: 0.0841
Early stopping:  154.5789576642158
Epoch: 003, Loss: 444.1395, Train: 0.0743, Test: 0.0745
Early stopping:  189.91627258312042
Epoch: 004, Loss: 426.9696, Train: 0.1531, Test: 0.1529
Early stopping:  174.98532282913413
Epoch: 005, Loss: 255.3884, Train: 0.2819, Test: 0.2844
Early stopping:  153.17976653559813
Epoch: 006, Loss: 322.4883, Train: 0.2399, Test: 0.2442
Early stopping:  84.76133832452409
Epoch: 007, Loss: 279.8001, Train: 0.1018, Test: 0.0947
Early stopping:  85.63415797421908
Epoch: 008, Loss: 249.2252, Train: 0.0794, Test: 0.0748
Early stopping:  73.09900908412011
Epoch: 009, Loss: 220.7157, Train: 0.2672, Test: 0.2642
Early stopping:  38.160551428691356
Epoch: 010, Loss: 159.8905, Train: 0.1605, Test: 0.1605
Early stopping:  61.352567268954076
Epoch: 011, Loss: 121.7891, Train: 0.1154, Test: 0.1106
Early stopping:  64.70814899048578
Epoch: 01

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 530 ms (started: 2024-10-16 21:43:28 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4742, Train: 0.2944, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 2.2547, Train: 0.2944, Test: 0.2943
Early stopping:  0.15525465998563992
Epoch: 003, Loss: 2.1587, Train: 0.2944, Test: 0.2943
Early stopping:  0.1617309272398368
Epoch: 004, Loss: 2.1243, Train: 0.2989, Test: 0.2990
Early stopping:  0.15746496134341417
Epoch: 005, Loss: 2.0498, Train: 0.3446, Test: 0.3391
Early stopping:  0.16386624540705622
Epoch: 006, Loss: 1.9798, Train: 0.3965, Test: 0.3859
Early stopping:  0.10489401807904415
Epoch: 007, Loss: 1.9162, Train: 0.4149, Test: 0.3973
Early stopping:  0.10011262414023951
Epoch: 008, Loss: 1.8514, Train: 0.4243, Test: 0.4060
Early stopping:  0.10750479258142429
Epoch: 009, Loss: 1.7870, Train: 0.4348, Test: 0.4184
Early stopping:  0.10344096988853532
Epoch: 010, Loss: 1.7208, Train: 0.4583, Test: 0.4407
Early stopping:  0.10235765843513896
Epoch: 011, Loss: 1.6510, Train: 0.4853, Test: 0.4644
Ear

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 439 ms (started: 2024-10-16 21:45:51 +00:00)


### Training rotulated base = 60% ❎

In [ ]:
rotulated_perc = 60

time: 643 µs (started: 2024-10-16 21:45:52 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 72.8832, Train: 0.2945, Test: 0.2940
Early stopping:  nan
Epoch: 002, Loss: 228.0030, Train: 0.1115, Test: 0.1111
Early stopping:  109.68625478547897
Epoch: 003, Loss: 540.5949, Train: 0.1161, Test: 0.1109
Early stopping:  238.23310531891005
Epoch: 004, Loss: 348.7739, Train: 0.1083, Test: 0.1086
Early stopping:  197.48980206121362
Epoch: 005, Loss: 386.3484, Train: 0.0788, Test: 0.0794
Early stopping:  175.57963221895253
Epoch: 006, Loss: 336.7390, Train: 0.0641, Test: 0.0649
Early stopping:  112.99521701431166
Epoch: 007, Loss: 281.9962, Train: 0.2781, Test: 0.2659
Early stopping:  97.82152749108369
Epoch: 008, Loss: 141.9578, Train: 0.2905, Test: 0.2889
Early stopping:  95.50216530683569
Epoch: 009, Loss: 148.7307, Train: 0.2909, Test: 0.2883
Early stopping:  110.28127416596163
Epoch: 010, Loss: 109.0522, Train: 0.2015, Test: 0.2090
Early stopping:  99.52583837842187
Epoch: 011, Loss: 66.4484, Train: 0.0953, Test: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 97.8822, Train: 0.0363, Test: 0.0380
Early stopping:  nan
Epoch: 002, Loss: 301.2881, Train: 0.2444, Test: 0.2470
Early stopping:  143.82974989284486
Epoch: 003, Loss: 211.7414, Train: 0.1323, Test: 0.1310
Early stopping:  101.94487001509788
Epoch: 004, Loss: 221.5844, Train: 0.2163, Test: 0.2070
Early stopping:  83.719946019617
Epoch: 005, Loss: 178.4780, Train: 0.1531, Test: 0.1395
Early stopping:  73.70583000886347
Epoch: 006, Loss: 276.8264, Train: 0.2794, Test: 0.2705
Early stopping:  50.03682290673954
Epoch: 007, Loss: 346.4602, Train: 0.1198, Test: 0.1179
Early stopping:  65.89069310080363
Epoch: 008, Loss: 382.2428, Train: 0.2841, Test: 0.2804
Early stopping:  84.5480254057158
Epoch: 009, Loss: 375.2343, Train: 0.1391, Test: 0.1429
Early stopping:  85.41857871542845
Epoch: 010, Loss: 351.5897, Train: 0.0794, Test: 0.0794
Early stopping:  41.7837375192993
Epoch: 011, Loss: 402.8260, Train: 0.2433, Test: 0.2390
Early stopping:  23.097270115238853
Epoch: 012, Los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 55.5327, Train: 0.2943, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 512.4736, Train: 0.1164, Test: 0.1162
Early stopping:  323.1060432764955
Epoch: 003, Loss: 372.3509, Train: 0.0737, Test: 0.0695
Early stopping:  234.09513738411567
Epoch: 004, Loss: 236.6483, Train: 0.1219, Test: 0.1222
Early stopping:  194.95745328083424
Epoch: 005, Loss: 252.0988, Train: 0.1147, Test: 0.1148
Early stopping:  169.88723773836486
Epoch: 006, Loss: 227.4258, Train: 0.0970, Test: 0.0930
Early stopping:  122.38727158294289
Epoch: 007, Loss: 146.4789, Train: 0.1030, Test: 0.1024
Early stopping:  81.1382367765386
Epoch: 008, Loss: 78.0980, Train: 0.2892, Test: 0.2915
Early stopping:  73.87837896119042
Epoch: 009, Loss: 50.7733, Train: 0.2355, Test: 0.2288
Early stopping:  88.66004942479682
Epoch: 010, Loss: 35.8971, Train: 0.1563, Test: 0.1554
Early stopping:  79.23171457012995
Epoch: 011, Loss: 25.4882, Train: 0.1928, Test: 0.1919
Early stopping:  48.45973540180207
Epoch: 012, Los

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 69.7165, Train: 0.1076, Test: 0.1103
Early stopping:  nan
Epoch: 002, Loss: 293.5366, Train: 0.2940, Test: 0.2940
Early stopping:  158.2646934973473
Epoch: 003, Loss: 452.5253, Train: 0.1185, Test: 0.1191
Early stopping:  192.31721515118787
Epoch: 004, Loss: 592.6407, Train: 0.0921, Test: 0.0879
Early stopping:  224.43649755170554
Epoch: 005, Loss: 354.8824, Train: 0.1221, Test: 0.1213
Early stopping:  194.37167778983212
Epoch: 006, Loss: 431.6418, Train: 0.2756, Test: 0.2730
Early stopping:  113.03127766437396
Epoch: 007, Loss: 244.1101, Train: 0.0524, Test: 0.0493
Early stopping:  128.50942747442213
Epoch: 008, Loss: 308.2895, Train: 0.1340, Test: 0.1457
Early stopping:  134.09236808035317
Epoch: 009, Loss: 252.2430, Train: 0.2767, Test: 0.2790
Early stopping:  77.70537613341129
Epoch: 010, Loss: 220.7060, Train: 0.1025, Test: 0.0950
Early stopping:  84.73528782649923
Epoch: 011, Loss: 196.3994, Train: 0.1123, Test: 0.1103
Early stopping:  41.85965544764048
Epoch: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 103.9409, Train: 0.1130, Test: 0.1120
Early stopping:  nan
Epoch: 002, Loss: 250.4422, Train: 0.0450, Test: 0.0471
Early stopping:  103.59204437486298
Epoch: 003, Loss: 331.3148, Train: 0.3087, Test: 0.3068
Early stopping:  115.25474575142303
Epoch: 004, Loss: 320.7521, Train: 0.1229, Test: 0.1239
Early stopping:  104.78712829078678
Epoch: 005, Loss: 399.8470, Train: 0.0707, Test: 0.0700
Early stopping:  112.38304743350012
Epoch: 006, Loss: 416.7873, Train: 0.0461, Test: 0.0434
Early stopping:  66.84156097050135
Epoch: 007, Loss: 399.0981, Train: 0.3013, Test: 0.2946
Early stopping:  44.116671871763444
Epoch: 008, Loss: 395.5215, Train: 0.1684, Test: 0.1667
Early stopping:  37.61082805136162
Epoch: 009, Loss: 392.9959, Train: 0.1123, Test: 0.1106
Early stopping:  9.329417485437716
Epoch: 010, Loss: 370.2709, Train: 0.3045, Test: 0.2980
Early stopping:  16.63788503553064
Epoch: 011, Loss: 324.4173, Train: 0.2355, Test: 0.2282
Early stopping:  31.208989620168012
Epoch: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 444 ms (started: 2024-10-16 21:47:41 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.4796, Train: 0.2943, Test: 0.2943
Early stopping:  nan
Epoch: 002, Loss: 2.2712, Train: 0.2943, Test: 0.2943
Early stopping:  0.14737067050319208
Epoch: 003, Loss: 2.1561, Train: 0.2945, Test: 0.2943
Early stopping:  0.16398403374456275
Epoch: 004, Loss: 2.1208, Train: 0.3106, Test: 0.3073
Early stopping:  0.1617375491900917
Epoch: 005, Loss: 2.0653, Train: 0.3629, Test: 0.3643
Early stopping:  0.16419509364065837
Epoch: 006, Loss: 2.0068, Train: 0.3949, Test: 0.3907
Early stopping:  0.09985075942851447
Epoch: 007, Loss: 1.9412, Train: 0.3883, Test: 0.3859
Early stopping:  0.08644665502710538
Epoch: 008, Loss: 1.8808, Train: 0.3881, Test: 0.3825
Early stopping:  0.09556796339180844
Epoch: 009, Loss: 1.8284, Train: 0.4006, Test: 0.3924
Early stopping:  0.0948983547380959
Epoch: 010, Loss: 1.7752, Train: 0.4295, Test: 0.4225
Early stopping:  0.0911842719401617
Epoch: 011, Loss: 1.7155, Train: 0.4652, Test: 0.4573
Early

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 451 ms (started: 2024-10-16 21:50:07 +00:00)


### Training rotulated base = 80% ❎

In [ ]:
rotulated_perc = 80

time: 367 µs (started: 2024-10-16 21:50:08 +00:00)


#### GCN Training

In [ ]:
for i in range(10):
    iteration(i,'GCN',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 66.5811, Train: 0.1131, Test: 0.1128
Early stopping:  nan
Epoch: 002, Loss: 430.3510, Train: 0.2802, Test: 0.2806
Early stopping:  257.2241343121809
Epoch: 003, Loss: 396.7217, Train: 0.0984, Test: 0.0935
Early stopping:  201.01917972208162
Epoch: 004, Loss: 661.6273, Train: 0.0332, Test: 0.0340
Early stopping:  244.9823035336127
Epoch: 005, Loss: 634.6266, Train: 0.0461, Test: 0.0454
Early stopping:  238.94851695423657
Epoch: 006, Loss: 577.0079, Train: 0.0740, Test: 0.0731
Early stopping:  120.0713565643617
Epoch: 007, Loss: 425.5712, Train: 0.2970, Test: 0.2931
Early stopping:  121.17718099745412
Epoch: 008, Loss: 303.9084, Train: 0.2919, Test: 0.2891
Early stopping:  151.70556820587274
Epoch: 009, Loss: 266.2439, Train: 0.1992, Test: 0.1859
Early stopping:  162.45010108779596
Epoch: 010, Loss: 216.3740, Train: 0.0963, Test: 0.0873
Early stopping:  144.85610480631155
Epoch: 011, Loss: 186.5578, Train: 0.0454, Test: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 77.4067, Train: 0.1019, Test: 0.1020
Early stopping:  nan
Epoch: 002, Loss: 357.6922, Train: 0.0471, Test: 0.0522
Early stopping:  198.19178281095006
Epoch: 003, Loss: 474.3578, Train: 0.2837, Test: 0.2823
Early stopping:  204.01839324394177
Epoch: 004, Loss: 456.1940, Train: 0.2443, Test: 0.2438
Early stopping:  183.31516172106396
Epoch: 005, Loss: 351.5122, Train: 0.1008, Test: 0.1003
Early stopping:  158.8198235819767
Epoch: 006, Loss: 339.5762, Train: 0.1940, Test: 0.1956
Early stopping:  64.01851699453388
Epoch: 007, Loss: 248.2382, Train: 0.2105, Test: 0.2132
Early stopping:  92.64757304069883
Epoch: 008, Loss: 205.4154, Train: 0.2348, Test: 0.2455
Early stopping:  97.72956749844586
Epoch: 009, Loss: 137.1826, Train: 0.0807, Test: 0.0726
Early stopping:  90.61377894866995
Epoch: 010, Loss: 140.4584, Train: 0.0897, Test: 0.0833
Early stopping:  84.14933268458549
Epoch: 011, Loss: 106.7614, Train: 0.1629, Test: 0.1593
Early stopping:  57.65357999908424
Epoch: 012,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 002, Loss: 577.9930, Train: 0.0734, Test: 0.0737
Early stopping:  354.43085426179175
Epoch: 003, Loss: 359.0671, Train: 0.0734, Test: 0.0680
Early stopping:  251.28760992144728
Epoch: 004, Loss: 256.8612, Train: 0.1112, Test: 0.1122
Early stopping:  209.14184731116646
Epoch: 005, Loss: 328.2101, Train: 0.0708, Test: 0.0692
Early stopping:  181.18349812648344
Epoch: 006, Loss: 309.4058, Train: 0.1374, Test: 0.1400
Early stopping:  124.02709980608027
Epoch: 007, Loss: 342.5348, Train: 0.1123, Test: 0.1190
Early stopping:  39.360922689647985
Epoch: 008, Loss: 343.1085, Train: 0.0644, Test: 0.0675
Early stopping:  35.808267752250465
Epoch: 009, Loss: 248.3663, Train: 0.1514, Test: 0.1536
Early stopping:  39.344080132666484
Epoch: 010, Loss: 196.5061, Train: 0.2959, Test: 0.3005
Early stopping:  64.05301830995579
Epoch: 011, Loss: 182.2905, Train: 0.3034, Test: 0.3061
Early stopping:  77.28486360436634
Epoch: 012, Loss: 154.3801, Train: 0.1726, Test: 0.1769
Early stopping:  74.361857

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 46.2419, Train: 0.1087, Test: 0.1088
Early stopping:  nan
Epoch: 002, Loss: 803.0211, Train: 0.1069, Test: 0.1049
Early stopping:  535.1236486015779
Epoch: 003, Loss: 548.5078, Train: 0.0258, Test: 0.0193
Early stopping:  385.08929938417884
Epoch: 004, Loss: 331.8402, Train: 0.0852, Test: 0.0833
Early stopping:  321.49199368926236
Epoch: 005, Loss: 156.9652, Train: 0.1090, Test: 0.0913
Early stopping:  304.4519564375853
Epoch: 006, Loss: 52.0546, Train: 0.1415, Test: 0.1264
Early stopping:  302.8787471022425
Epoch: 007, Loss: 17.1891, Train: 0.1510, Test: 0.1406
Early stopping:  220.0728653305294
Epoch: 008, Loss: 10.4900, Train: 0.1704, Test: 0.1633
Early stopping:  135.29951157171618
Epoch: 009, Loss: 7.9110, Train: 0.1717, Test: 0.1746
Early stopping:  62.94650185985652
Epoch: 010, Loss: 6.0298, Train: 0.1628, Test: 0.1667
Early stopping:  19.099414941686128
Epoch: 011, Loss: 4.7755, Train: 0.1602, Test: 0.1627
Early stopping:  4.917994878962689
Epoch: 012, Loss: 3

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Epoch: 001, Loss: 55.1530, Train: 0.0777, Test: 0.0754
Early stopping:  nan
Epoch: 002, Loss: 242.6806, Train: 0.1126, Test: 0.1156
Early stopping:  132.60204254400875
Epoch: 003, Loss: 324.1827, Train: 0.2963, Test: 0.2937
Early stopping:  137.95299453981295
Epoch: 004, Loss: 285.5003, Train: 0.1442, Test: 0.1395
Early stopping:  119.22524268920957
Epoch: 005, Loss: 335.7411, Train: 0.2334, Test: 0.2245
Early stopping:  114.15419572429424
Epoch: 006, Loss: 271.6758, Train: 0.0906, Test: 0.0822
Early stopping:  38.19923631237197
Epoch: 007, Loss: 354.8021, Train: 0.1489, Test: 0.1474
Early stopping:  34.79950456330609
Epoch: 008, Loss: 312.6161, Train: 0.3031, Test: 0.2931
Early stopping:  34.36674400412198
Epoch: 009, Loss: 305.7869, Train: 0.1636, Test: 0.1593
Early stopping:  31.526301097366343
Epoch: 010, Loss: 307.1459, Train: 0.2398, Test: 0.2347
Early stopping:  29.613719971338313
Epoch: 011, Loss: 235.2035, Train: 0.2681, Test: 0.2630
Early stopping:  43.003448696537845
Epoch: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 518 ms (started: 2024-10-16 21:52:42 +00:00)


#### Second Network - GAT

In [ ]:
for i in range(10):
    iteration(i,'GAT',str(rotulated_perc)+'%')


=================== MODEL 0 ===================

Epoch: 001, Loss: 2.5506, Train: 0.1177, Test: 0.1179
Early stopping:  nan
Epoch: 002, Loss: 2.3214, Train: 0.2942, Test: 0.2931
Early stopping:  0.16208447249313326
Epoch: 003, Loss: 2.1771, Train: 0.2943, Test: 0.2942
Early stopping:  0.18837555777004578
Epoch: 004, Loss: 2.1539, Train: 0.2970, Test: 0.2954
Early stopping:  0.1823264740585029
Epoch: 005, Loss: 2.0988, Train: 0.3190, Test: 0.3169
Early stopping:  0.18189517321614337
Epoch: 006, Loss: 2.0155, Train: 0.3774, Test: 0.3668
Early stopping:  0.11265235905501024
Epoch: 007, Loss: 1.9623, Train: 0.4029, Test: 0.3912
Early stopping:  0.09113934759687246
Epoch: 008, Loss: 1.9161, Train: 0.4173, Test: 0.3997
Early stopping:  0.09727552699981093
Epoch: 009, Loss: 1.8601, Train: 0.4255, Test: 0.4155
Early stopping:  0.0917943548574747
Epoch: 010, Loss: 1.7982, Train: 0.4266, Test: 0.4150
Early stopping:  0.08499114633131459
Epoch: 011, Loss: 1.7373, Train: 0.4322, Test: 0.4189
Earl

In [ ]:
# Saving dataframes_big
for i in range(10):
    df_list[i][0].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_acc.pkl") # acuracia
    df_list[i][1].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_pre.pkl") # precision
    df_list[i][2].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_rec.pkl") # recall
    df_list[i][3].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_f1.pkl")  # f1-score
    df_list[i][4].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_sup.pkl") # support
    df_list[i][5].to_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/dataframes_big/"+str(i)+"/df_time.pkl") # time

time: 471 ms (started: 2024-10-16 21:55:20 +00:00)
